<a href="https://colab.research.google.com/github/vincent-arese/OC-P04VA/blob/main/P4_DataSet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Config Env de travail

##### pip install

In [99]:
# Prepa Env PIP
# librairie pour combiner Pyhton & SQL https://towardsdatascience.com/fugue-and-duckdb-fast-sql-code-in-python-e2e2dfc0f8eb
#!pip install -U fugue[duckdb,sql] 
# !pip install pandas-bokeh

#  TO DEL    !pip install -U dash 

#! pip list -v
# ! pip list -v | grep [Pp]an 
# ! pip list -v | grep [Pp]lot
# ! pip list -v | grep [Ss]ea

# ! pip install --upgrade pandas

##### Librairies

In [100]:
# librairies Pythons 
import numpy as np
import pandas as pd
from pandas.core.groupby.generic import DataFrameGroupBy
pd.set_option('display.max_columns',None)  #pd.set_option('max_columns', 10) limiter le nbr de colonnes visualisé à 10 /none (=> scrollbar)
pd.set_option('display.max_rows', 40)
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
# import pandas_bokeh
#import fugue_duckdb
#from fugue_notebook import setup ;setup()


#  Importer les données en Pandas dataframe
#aideDataPATH='/content/drive/MyDrive/DataSc/P4VA/'
DataPATH='https://raw.githubusercontent.com/vincent-arese/OC-P04VA/main/FAOdata/'

#### Fonctions utilisés pour le Projet

##### Search NaN  SearchNaN(DataFrame,Option)

In [101]:
#Fonction ChercheNaN : Recherche des NaN
def SearchNaN(DataFrame,Option):
  """ChercheNaN : Recherche des NaN
  
  Option : 
  1: Total nombre de NaN par colonne
  2: Afficher les lignes ayant au moins 1 NaN
  3: Nomnbre de lignes ayant 1 NaN
  """
  
  if Option==1:
   return print('Nombre de NaN par colonne (total lignes',len(DataFrame),'):/n' ,DataFrame.isnull().sum(axis=0)) #  Option1 : Total nombre de NaN par colonne
  elif Option==2:
    return DataFrame[DataFrame.isnull().any(axis=1)]  #Afficher les lignes ayant au moins 1 NaN
  elif Option==3: 
    return print('Nombre de ligne(s) ayant au moins 1 NaN :', len(DataFrame[DataFrame.isnull().any(axis=1)]))
  else:
    print("SearchNaN(DataFrame,Option) Error :","Choisir Option :\n","\n 1: Total nombre de NaN par colonne","\n 2: Total nombre de NaN par colonne","\n 3: Nomnbre de lignes ayant 1 NaN")

# DEBUG  recherche NaN
# temp=geo
# temp.isnull().sum(axis=0) #  Option1 : Total nombre de NaN par colonne
# test=temp[temp.isnull().any(axis=1)] ; test #Afficher les lignes ayant au moins 1 NaN
# len(test)
# test.Zone.unique()

##### SearchListe(df,liste)

In [102]:
# Recherche une liste dans un DataFrame
def SearchListe(df,liste):
  """Recherche une liste dans un DataFrame et renvoie Df avec les Valeur pour masque SearchListe(dx,[2,4,7]) - liste=[2,4,7]/ SearchListe(dx,liste) accepte liste unique liste=[1]"""
  return df[df.isin(liste)].dropna(thresh=1)

# Exemple : 
# dx = pd.DataFrame(np.random.randint(10, size=(10, 3)),
#                      columns=['A', 'B', 'C'])
# dx
# SearchListe(dx,[2,4,7])
# liste=[2,4,7]
# SearchListe(dx,liste)
# SearchListe(dx,l)

##### MultiDOWN ( Conversion multi-index  à index simple)

In [103]:
#Fonction multiDOWN : Conversion df_MultiIndex to df_indexSimple
def multiDOWN(DataFrame):
  """Fonction multiDOWN(DataFrame) : Conversion DataFrame_MultiIndex to DataFrame_indexSimple"""
  DataFrame=DataFrame.columns=DataFrame.columns.to_flat_index() # colonnes => concatener le multi index en un index simple https://datascientyst.com/flatten-multiindex-in-pandas/
  DataFrame=DataFrame.reset_index() # Conversion Index en colonnes simple.
  return DataFrame

  # probleme  d'execution 

##### ColMove(df,ColName,ColIndex)

In [104]:
# Deplacer une colonne dans un dataframe 
def ColMove(DataFrame,ColName,ColIndex):
  """Fonction  ColMove(DataFrame,ColName,ColIndex)  
   Deplace un colonne dans un DataFrame  - ColName : nom colonne  - ColIndex : rang index souhaité (ddbut df 0 !)
  """ 
  DataFrame.insert(ColIndex,'Xcol',DataFrame[ColName])
  DataFrame.drop(ColName, axis=1, inplace=True)
  DataFrame.rename(columns={'Xcol' : ColName},inplace=True)
  return DataFrame

##### np.Array  LostElement(Array1,Array2)


In [105]:
# Rechercher un élément perdu à partir d’un tableau dupliqué 
def LostElement(A,B):
  """Rechercher un élément perdu à partir d’un tableau dupliqué
    LostElement(ArrayA,ArrayB)  ou LostElement(df.A.unique(),df.B.unique())
    Compare deux np.array !!!!
      =>  DifférenceA=>B  : Valeur incluse dans A mais Non dans B 
  """

  return print('ArrayA:',A,'- Nb val:',len(A),'\nArrayB:',B,'- Nb val:',len(B),'\nDifférenceA=>B',np.setdiff1d(A, B),'\nDifférenceB=>A',np.setdiff1d(B, A))
 

##### ReduceMemUsage reduce_mem_usage(df)

In [106]:
# Drastically reduce df RAM usage ! 
#I don't know who the original author of this function but many thanks to him ;) 
#source : https://towardsdatascience.com/how-to-learn-from-bigdata-files-on-low-memory-incremental-learning-d377282d38ff
def reduce_mem_usage(df):
    """ 
    iterate through all the columns of a dataframe and 
    modify the data type to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print(('Memory usage of dataframe is {:.2f}' 
                     'MB').format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max <\
                  np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max <\
                   np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max <\
                   np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max <\
                   np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max <\
                   np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max <\
                   np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')
    end_mem = df.memory_usage().sum() / 1024**2
    print(('Memory usage after optimization is: {:.2f}' 
                              'MB').format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) 
                                             / start_mem))
    
    return df

    # Attention ne fct pas si le df à optimizer contient un type 'category'
  #Code ...........
    # df.info(memory_usage=True)
    # DataFrame.memory_usage(index=True, deep=False)

###### dtype Min-Max

In [107]:
pd.DataFrame.from_dict(
{'np.int8:':[np.iinfo(np.int8).min, np.iinfo(np.int8).max],
'np.int16:':[np.iinfo(np.int16).min, np.iinfo(np.int16).max],
'np.int32:':[np.iinfo(np.int32).min, np.iinfo(np.int32).max],
'np.int64:':[np.iinfo(np.int64).min, np.iinfo(np.int64).max],
'np.float16:':[np.finfo(np.float16).min, np.finfo(np.float16).max],
'np.float32:':[np.finfo(np.float32).min, np.finfo(np.float32).max],
'np.float64:':[np.finfo(np.float64).min, np.finfo(np.float64).max]})

## info complementaire https://towardsdatascience.com/reducing-memory-usage-in-pandas-with-smaller-datatypes-b527635830af

,np.int8:,np.int16:,np.int32:,np.int64:,np.float16:,np.float32:,np.float64:
0,-128,-32768,-2147483648,-9223372036854775808,-65504.0,-3.402823e+38,-1.797693e+308
1,127,32767,2147483647,9223372036854775807,65504.0,3.402823e+38,1.797693e+308


##### Change dtype Category to Object : dtypeCat2Obj(df)

In [108]:
def dtypeCat2Obj(df):
    """  iterate through all the columns of a dataframe and 
    modify the Dtype Catergory to Object (Str)       
    """
     
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type == 'category':
             df[col] = df[col].astype(str)    
           
    return df

##### ListeColUnique(df)

In [109]:
def ListeColUnique(df):
  """ Itere sur les colonnes d'un DataFrame(df) et listes les valeurs unique de chaque colonne"""
  ListeCol=df.columns
  for col in ListeCol: 
    return  print(col,':',len(df[col].unique()),': \n',df[col].sort_values(ascending=True,na_position='first').unique(),'\n______________\n')

##### Différences entre deux df get_different_rows(source_df, new_df)

In [110]:
def get_different_rows(source_df, new_df):
    """Returns just the rows from the new dataframe that differ from the source dataframe"""
    merged_df = source_df.merge(new_df, indicator=True, how='outer')
    changed_rows_df = merged_df[merged_df['_merge'] == 'right_only']
    return changed_rows_df.drop('_merge', axis=1)


#Trouver les lignes peu communes entre deux DataFrames
# pd.concat([SecuAlimGeo,SecuAlim]).drop_duplicates(keep=False)

# #Trouver les lignes peu communes entre deux DataFrames
# pd.concat([SecuAlimGeo,SecuAlim]).drop_duplicates(keep=False)

##### PercentCumul(df,Col) Calcul % cumulé

In [111]:
# Tri ordre decroissant puis ajoute Colonnne avec Calcul % cumulé pour identifier 20/80 ou 80/20 
def PercentCumul(df,Col):
  """ Tri ordre decroissant puis ajoute Colonnne Col_cum% avec Calcul % cumulé pour identifier facilement 20/80 ou 80/20 """
  df=df.sort_values(by=[Col],ascending=False)
  df[Col+'cum_%']=(df[Col].cumsum() / df[Col].sum()) * 100
  # df=df.sort_values(by=[Col+'cum_%'],ascending=False)
  return df


# Cf cumsum() pour  SommeCumulée
 #https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.cumsum.html 


##### InfoALL

In [112]:
def InfoALL(df):
  """ SearchNaN(df,1) & df.info() """
  return print(SearchNaN(df,1),df.info(),'\n------------------\n')

In [113]:
# df # 4114 
# df[df.isnull()] #4114 rows
# df[~df.isnull()] # 4114 rows 
# SearchNaN(df,2) #562 rows × 17 columns


# Nouvelle section

 RAM 1.1/ 12go

# Data set FAO

## <font color='Orange'>Géopolitique 2000-2019</font>

### Population Mondiale 

In [114]:
# Population Mondiale RAW dataSet FAO  source : https://www.fao.org/faostat/fr/#data/FBSH
popI=pd.read_csv(DataPATH+'FAOSTAT_pop1961-2013.csv',sep=',')
popII=pd.read_csv(DataPATH+'FAOSTAT_pop2010-2019.csv',sep=',')
popII.rename(columns={'Code zone (FAO)':'Code zone'},inplace=True) # Renommer la colonne qui servira de fK pour la jonction 

reduce_mem_usage(popI) # df RAM usage opt with best possible .astype(" .....")
reduce_mem_usage(popII) # df RAM usage opt with best possible .astype(" .....")


#Debug
# popI.columns==popII.columns  # Ok
# popI[(popI.Année<2010)].Année.unique()   # [1961,2009]
# popII.Année.unique()  # [2010,2019]
#pop=pd.merge(popI, popII, how='outer')

# Restriction des doublons & Merge des deux CSV
pop=pd.merge(popI[popI.Année<=2009], popII, how='outer')
# pop[(pop.Zone=='Afghanistan') & (pop.Année>=2005)] # Verification pas de doublon


# Projection : Nettoyage Data Frame pop 

pop=pop.iloc[:,[i for i in range(2,12)]] # supression rapide des colonnes inutiles

# pop.columns
"""Index(['Code Domaine', 'Domaine', 'Code zone', 'Zone', 'Code Élément',
       'Élément', 'Code Produit', 'Produit', 'Code année', 'Année', 'Unité',
       'Valeur', 'Symbole', 'Description du Symbole', 'Code zone (FAO)'],
      dtype='object')"""

pop.drop(columns=['Code année','Code Élément','Élément','Unité', 'Code Produit', 'Produit'],inplace=True) # supression colonnes inutiles restantes

pop.rename(columns={'Code zone':'CodeZone','Valeur':'Population'},inplace=True) 
pop.Population=(pop.Population*1000) # Convertion Population  (car unité PopI & PopII :  millier hab)
pop['PopulationM']=pop.Population/10**6  # Creation Pop Million
pop=pop[pop.Année>=2000]  # Restriction conserver que les données depuis 2000

pop.Zone=pop.Zone.astype("category") # pop.memory_usage(deep=True) #Optimzation RAM usage!
pop.Population =pop.Population .astype(np.int32)
pop.PopulationM =pop.PopulationM .astype(np.float32)
pop.head(1)
# Recherche des NaN
# pop.isnull().sum(axis=0) # recherche & sommme des Nan ds chaque colonne   # OK pas de NaN
# pop[pop.isnull().any(axis=1)] # Visuliser les lignes contenant au moins un NaN

# DEBUG  recherche origine CodeZone Manquant suite à la jointure  
# print(pop[pop.isnull().any(axis=1)].Zone.unique(),'Nombre :',len(pop[pop.isnull().any(axis=1)].Zone.unique())) # Visuliser les lignes contenant au moins un NaN  / listes des pays concernés 

# pop.info()
# SearchNaN(pop,1)  # Ok pas de NAN 
# len(pop)

Memory usage of dataframe is 0.92MB
Memory usage after optimization is: 0.20MB
Decreased by 78.8%
Memory usage of dataframe is 0.19MB
Memory usage after optimization is: 0.05MB
Decreased by 76.2%


,CodeZone,Zone,Année,Population,PopulationM
39,2,Afghanistan,2000,20595000,20.594999


In [115]:
pop


,CodeZone,Zone,Année,Population,PopulationM
39,2,Afghanistan,2000,20595000,20.594999
40,2,Afghanistan,2001,21348000,21.348000
41,2,Afghanistan,2002,22203000,22.202999
42,2,Afghanistan,2003,23116000,23.115999
43,2,Afghanistan,2004,24019000,24.018999
...,...,...,...,...,...
9710,181,Zimbabwe,2015,13815000,13.815000
9711,181,Zimbabwe,2016,14030000,14.030000
9712,181,Zimbabwe,2017,14237000,14.237000
9713,181,Zimbabwe,2018,14439000,14.439000


### PIB par habitant, (US$ PPA internationaux constants de 2015)

In [116]:
# PIB par habitant, (US$ PPA internationaux constants de 2015) dataSet FAO  source : https://www.fao.org/faostat/fr/#data/PD
pib=pd.read_csv(DataPATH+'FAOSTAT_PIBhab-1970-2019.csv',sep=',',usecols = [i for i in range(2,12)]) 


# Nettoyage PIB 
# pibRAW.columns """Index(['Code zone (FAO)', 'Zone', 'Code ?l?ment', '?l?ment', 'Code Produit','Produit', 'Code ann?e', 'Ann?e', 'Unit?', 'Valeur'], dtype='object')"""
pib.drop(columns=['Code ?l?ment', '?l?ment', 'Code ann?e', 'Unit?', 'Code Produit','Produit'],inplace=True) # supression colonnes inutiles restantes
pib.rename(columns={'Code zone (FAO)':'CodeZone','Ann?e':'Année','Valeur':'USDparHab'},inplace=True) # Renommer la colonne qui servira de fK pour la jonction 
pib=pib[pib.Année>=2000]  # Restriction conserver que les données depuis 2000


#Optimzation RAM usage! 
# pib.Zone=pib.Zone.astype("category") # pop.memory_usage(deep=True)
reduce_mem_usage(pib)

pib.head(1)

#DEGUG :  Pas de NaN ok 
# pib.info()


Memory usage of dataframe is 0.16MB
Memory usage after optimization is: 0.07MB
Decreased by 54.0%


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,CodeZone,Zone,Année,USDparHab
30,2,Afghanistan,2000,62.53125


In [117]:
pib.info()
SearchNaN(pib,1)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4195 entries, 30 to 10013
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   CodeZone   4195 non-null   int16   
 1   Zone       4195 non-null   category
 2   Année      4195 non-null   int16   
 3   USDparHab  4195 non-null   float16 
dtypes: category(1), float16(1), int16(2)
memory usage: 75.3 KB
Nombre de NaN par colonne (total lignes 4195 ):/n CodeZone     0
Zone         0
Année        0
USDparHab    0
dtype: int64


### Liste Pays

In [118]:
# Liste des Pays avec CodeZone FAO  & AlphaISO3 pour plot sur map Monde 
# source : https://www.fao.org/faostat/fr/#data/FS  Rubrique : Définitions et Standards  Formaté sous Excel
PAYS=pd.read_excel('https://github.com/vincent-arese/OC-P04VA/blob/main/FAOdata/PAYS%20_FAO.xlsx?raw=true', sheet_name='PaysFAO')   # DataSet pour plot map monde 
PAYS.rename(columns={'Code_ZoneContinent':'CodeZoneContinent','Code_Pays':'CodeZone','Pays':'Zone'},inplace=True)

# PAYS  optimisation dtype pour la RAM
# cols = ['continent', 'ZoneContinent', 'Zone', 'AlphaISO3']
# PAYS[cols] = PAYS[cols].astype('category')
colsnum = ['CodeContinent', 'CodeZoneContinent', 'CodeZone']
PAYS[colsnum] = PAYS[colsnum].astype(np.int16)

# Trier le df par colonne
PAYS.sort_values(by=['CodeZone'],inplace=True)
# PAYS.sort_values(by=['CodeZone'],inplace=True)
# PAYS.sort_values(by=['continent','CodeZone'],inplace=True)
PAYS
PAYScol=PAYS.columns

#DEBUG 
# SearchNaN(PAYS,1)  - ok pas de NaN
#  PAYS.info()

In [119]:
PAYS

,CodeContinent,continent,CodeZoneContinent,ZoneContinent,CodeZone,Zone,AlphaISO3,Note
10,5300,Asie,5305,Asie occidentale,1,Arménie,ARM,Ex URSS
0,5300,Asie,5303,Asie méridionale,2,Afghanistan,AFG,NaN
2,5400,Europe,5403,Europe méridionale,3,Albanie,ALB,NaN
3,5100,Afrique,5103,Afrique septentrionale,4,Algérie,DZA,NaN
5,5100,Afrique,5102,Afrique centrale,7,Angola,AGO,NaN
...,...,...,...,...,...,...,...,...
178,5100,Afrique,5103,Afrique septentrionale,276,Soudan ex,SDN,Soudan ex
177,5100,Afrique,5101,Afrique orientale,277,Soudan du Sud,SSD,Update1
136,5300,Asie,5305,Asie occidentale,299,Palestine,PSE,Update2
38,5300,Asie,5302,Asie orientale,351,Chine,CHN,NaN


In [120]:
#Enlever la Colone Note pour la suite
PAYS.drop(columns='Note',inplace=True)

In [121]:
 PAYS.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 204 entries, 10 to 41
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   CodeContinent      204 non-null    int16 
 1   continent          204 non-null    object
 2   CodeZoneContinent  204 non-null    int16 
 3   ZoneContinent      204 non-null    object
 4   CodeZone           204 non-null    int16 
 5   Zone               204 non-null    object
 6   AlphaISO3          204 non-null    object
dtypes: int16(3), object(4)
memory usage: 9.2+ KB


### Recap Géopolitique

In [122]:
# Recap Data Géopolitique : Population, PIB par Hab (USD 20015), Liste PAYS

#### DEBUG 
# Erreur sur les merge  Geo avec Pays qui ont changés ( par Ex Yougoslavie .....Thecoslovaquie .... ) qui génèrent des NAN 
# Ne pas faire de merge sur CodeZOne mais sur ISo ALPHA ( qui est unique (Pays avec la plus grande pop des Pays dissous))
# Attention Certain payas avec Année 0  : abscence données Pop  et USD/hab ...
#####


PAYS_ISO=PAYS.drop(['CodeContinent', 'continent', 'CodeZoneContinent', 'ZoneContinent', 'Zone'], axis=1)

m1=pop.merge(PAYS_ISO).drop(['CodeZone','Zone'],axis=1)
m2=pib.merge(PAYS_ISO).drop(['CodeZone','Zone'],axis=1)

geo=m1.merge(m2).merge(PAYS,how='right')
geo.fillna(0,inplace=True) # Attention Certain payas avec Année 0  : abscence données Pop  et USD/hab ..
geo.Année=geo.Année.astype(np.int16)
ColMove(geo,'CodeZone',0)
ColMove(geo,'AlphaISO3',1)
ColMove(geo,'Zone',2)

reduce_mem_usage(geo)


#DEBUG
# SearchNaN(geo,1)  #OK
# SearchNaN(pop.merge(PAYS),1) # OK pas de NaN
#geo.info()   # OK Dtype optimisé

Memory usage of dataframe is 0.22MB
Memory usage after optimization is: 0.12MB
Decreased by 44.4%


,CodeZone,AlphaISO3,Zone,Année,Population,PopulationM,USDparHab,CodeContinent,continent,CodeZoneContinent,ZoneContinent
0,1,ARM,Arménie,2000,3.076000e+06,3.076172,52.28125,5300,Asie,5305,Asie occidentale
1,1,ARM,Arménie,2001,3.060000e+06,3.060547,52.90625,5300,Asie,5305,Asie occidentale
2,1,ARM,Arménie,2002,3.047000e+06,3.046875,51.56250,5300,Asie,5305,Asie occidentale
3,1,ARM,Arménie,2003,3.036000e+06,3.035156,53.43750,5300,Asie,5305,Asie occidentale
4,1,ARM,Arménie,2004,3.026000e+06,3.025391,61.62500,5300,Asie,5305,Asie occidentale
...,...,...,...,...,...,...,...,...,...,...,...
3724,357,CHN,Chine (exc. Hong-Kong et Macao),2017,1.421022e+09,1421.000000,97.37500,5300,Asie,5302,Asie orientale
3725,357,CHN,Chine (exc. Hong-Kong et Macao),2018,1.459378e+09,1459.000000,103.00000,5300,Asie,5302,Asie orientale
3726,357,CHN,Chine (exc. Hong-Kong et Macao),2018,1.427648e+09,1428.000000,103.00000,5300,Asie,5302,Asie orientale
3727,357,CHN,Chine (exc. Hong-Kong et Macao),2019,1.465634e+09,1466.000000,100.18750,5300,Asie,5302,Asie orientale


In [221]:
geo.Année.sort_values().unique()

array([   0, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009,
       2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019],
      dtype=int16)

##### Verif Recap

In [123]:
# Attention Certain Pays avec Année 0  : abscence données Pop  et USD/hab ...
geo[geo.Année==0]
geo[geo.Année==0].Zone.unique()  #13 Pays  ['Bahreïn', 'Bhoutan', 'Guinée équatoriale', 'Micronésie (États fédérés de)', 'Porto Rico', ..., 'Somalie', 'Chine, Taiwan Province de', 'Tonga', 'Soudan du Sud', 'Palestine']
geo[geo.Année==0].CodeZone.unique() # 13 pays [ 13,  18,  61, 145, 177, 178, 186, 200, 201, 214, 219, 277, 299],
geo[geo.Année!=0]


# get_different_rows(geo[geo.Année==0], geo[geo.Année!=0])
get_different_rows(geo[geo.Année!=0], geo)


# DEBUG: 
# df=geo
# ListeCol=df.columns
# for col in ListeCol:
#  print(col,':',len(df[col].unique()),': \n',df[col].sort_values(ascending=True,na_position='first').unique(),'\n______________\n')
# geo[geo.Année!=0].CodeZone.sort_values(ascending=True,na_position='first').unique()

,CodeZone,AlphaISO3,Zone,Année,Population,PopulationM,USDparHab,CodeContinent,continent,CodeZoneContinent,ZoneContinent
3714,13,BHR,Bahreïn,0,0.0,0.0,0.0,5300,Asie,5305,Asie occidentale
3715,18,BTN,Bhoutan,0,0.0,0.0,0.0,5300,Asie,5303,Asie méridionale
3716,61,GNQ,Guinée équatoriale,0,0.0,0.0,0.0,5100,Afrique,5102,Afrique centrale
3717,76,PSE,Palestine,0,0.0,0.0,0.0,5300,Asie,5305,Asie occidentale
3718,145,FSM,Micronésie (États fédérés de),0,0.0,0.0,0.0,5300,Asie,5304,Asie du Sud-Est
3719,177,PRI,Porto Rico,0,0.0,0.0,0.0,5200,Amériques,5206,Caraïbes
3720,178,ERI,Érythrée,0,0.0,0.0,0.0,5100,Afrique,5101,Afrique orientale
3721,179,QAT,Qatar,0,0.0,0.0,0.0,5300,Asie,5305,Asie occidentale
3722,186,SCG,Serbie-et-Monténégro,0,0.0,0.0,0.0,5400,Europe,5403,Europe méridionale
3723,200,SGP,Singapour,0,0.0,0.0,0.0,5300,Asie,5304,Asie du Sud-Est


## <font color='Orange'> Sécurité Alimentaire  2010-2020</Font>

### Import data FAO Secu Alim

In [272]:
#FAO  Securité Alimentaire   https://www.fao.org/faostat/fr/#data/FS 
# Rubrique Toutes les données 	https://fenixservices.fao.org/faostat/static/bulkdownloads/Données_de_la_sécurité_alimentaire_F_Toutes_les_Données_(Normalisé).zip
#Probleme encodage donc import des csv dans Excel PowerQuery Normalisation

SecuAlimRAW=pd.read_excel('https://github.com/vincent-arese/OC-P04VA/blob/main/FAOdata/FAO_SecuAlimNORM.xlsx?raw=true', sheet_name='SecuAlimDataNorm')
SecuAlimlisteCode=pd.read_excel('https://github.com/vincent-arese/OC-P04VA/blob/main/FAOdata/FAO_SecuAlimNORM.xlsx?raw=true', sheet_name='SecuAlimListeProduit')

# FAOSecuAlimDataNorm.zip 
# usecols = [i for i in range(10)

### Nettoyage & formatage data

In [273]:
# Supprimer les lignes sans Valeur : Colonne 'Valeur' => 5159 NaN sur   148060 entries
# SecuAlim.info()
# SearchNaN(SecuAlim,1)
SecuAlimRAW=SecuAlimRAW.dropna() # drop Row ayant au moins un NaN

# Restriction  les CodeZone correspondant à des agrégats ( continent, groupe de pays ........... ) =>  CodeZone>=400 ! 
''' 'Pays les moins développés' 'Pays sans littoral en voie de développement' 'Petites îles en voie de Développement'  'Pays à faible revenu et à déficit vivrier' 'Pays à faible revenu'  'Pays à revenu moyen' 'High-income economies'  'Upper-middle-income economies' '''

SecuAlim=SecuAlimRAW.loc[SecuAlimRAW['CodeZone'] <= 400].sort_values(by=['CodeZone'])
# SecuAlim.CodeZone.sort_values(ascending=True).unique() # Verif code OK 

#### Conversion CodeAnnée en AnnéeREF

In [274]:
# Converions Période/Année en AnnéeREF 

# SecuAlim.loc[SecuAlim.CodeAnnée <=3000,'CodeAnnée'].unique()
# SecuAlim.loc[SecuAlim.CodeAnnée >=3000,'CodeAnnée'] : TEST pour isoler les Années simple des Periodes) 
# SecuAlim.loc[SecuAlim.CodeAnnée >=3000,'AnnéeREF'].astype(str).str[:4].astype(int)  Isoler la Année_début de la periode
# AnnéeREF =(Année_début +Année_fin)/2 


SecuAlim.rename(columns={'CodeAnnée':'AnnéeREF'},inplace=True)
SecuAlim.loc[SecuAlim.AnnéeREF >=3000,'AnnéeREF']=((SecuAlim.loc[SecuAlim.AnnéeREF >=3000,'AnnéeREF'].astype(str).str[:4].astype(int)+SecuAlim.loc[SecuAlim.AnnéeREF >=3000,'AnnéeREF'].astype(str).str[4:8].astype(int))/2).astype(int)

# DEBUG
# SecuAlim.info(memory_usage=True)
# SearchNaN(SecuAlim ,1) # OK pas de NaN

#### Selection  des Codes Produits à conserver 

##### liste des Code Produits dispo

In [275]:
#Sélection des Codes produits à conserver 
SecuAlimlisteCode['ListeProduit']=SecuAlimlisteCode.CodeProduit.astype(str)+'-'+SecuAlimlisteCode.Produit # Creation Colonne listeProduit pour filtrage 
SecuAlimlisteCode
SecuAlimlisteCode.ListeProduit.unique()  # Print listeProduit pour filtrage 

array(['21010-Suffisance des apports énergétiques alimentaires moyens (%) (moyenne sur 3 ans)',
       '21011-Valeur moyenne de la production alimentaire ($I constants par personne) (moyenne sur 3 ans)',
       '21012-Part des céréales, des racines et des tubercules dans les apports énergétiques alimentaires (%) (moyenne sur 3 ans)',
       '21013-Disponibilités protéiques moyennes (g/personne/jour) (moyenne sur 3 ans)',
       '21014-Disponibilités protéines moyennes d’origine animale (g/personne/jour) (moyenne sur 3 ans)',
       '21016-Densité du réseau ferroviaire (pour 100 km carrés de surface totale du pays)',
       '21025-Pourcentage des enfants de moins de 5 ans présentant un retard de croissance (estimation modélisée) (%)',
       '21026-Pourcentage des enfants de moins de 5 ans émaciés (%)',
       '21030-Variabilité de la production alimentaire par habitant ($I par personne constant 2004-06)',
       '21031-Variabilité des disponibilités alimentaires par habitant (Kcal/pers

##### liste des Code Produits retenus

In [276]:
# Restriction : Choix des CodeProduit à conserver  cf. ListeProduit au-dessus
SelectedCodeProduit=[210011,21056,21057] ;SelectedCodeProduit
# SelectedCodeProduit=[21013,21014,210011,21056,21057,21061] ;SelectedCodeProduit
SecuAlim_selected=SecuAlimlisteCode[SecuAlimlisteCode.CodeProduit.isin(SelectedCodeProduit)]
# SecuAlim_selected.ListeProduit.unique() #Verification restriction
SecuAlim_selected

,CodeProduit,Produit,Unité,Description,ListeProduit
23,21056,Besoins énergétiques alimentaires minimaux (kc...,Kcal/personne/jour,"In a specified age/sex category, MDER is the m...",21056-Besoins énergétiques alimentaires minima...
24,21057,Besoins énergétiques alimentaires moyens (kcal...,Kcal/personne/jour,The ADER is a proper normative reference for a...,21057-Besoins énergétiques alimentaires moyens...
31,210011,Nombre de personnes sous-alimentées (millions)...,millions,Estimated number of people at risk of undernou...,210011-Nombre de personnes sous-alimentées (mi...


In [277]:
# Restruction du df SecuAlim
SecuAlim=SecuAlim[SecuAlim.CodeProduit.isin(SelectedCodeProduit)]
SecuAlim.CodeProduit.sort_values(ascending=True).unique()#Verification restriction

#dEBUG ok 
# SearchNaN(SecuAlim,1) #PAs de NaN
# SearchListe(SecuAlim,[0])  #Pas de 0 

array([ 21056,  21057, 210011])

#### Nettoyage Valeurs et optimisation dtype

In [278]:
SecuAlim.info() #10340 entries
SecuAlim.describe(include='all')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10340 entries, 6597 to 23170
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   CodeZone     10340 non-null  int64 
 1   CodeProduit  10340 non-null  int64 
 2   AnnéeREF     10340 non-null  int64 
 3   Valeur       10340 non-null  object
 4   Symbole      10340 non-null  object
dtypes: int64(3), object(2)
memory usage: 484.7+ KB


,CodeZone,CodeProduit,AnnéeREF,Valeur,Symbole
count,10340.000000,10340.00000,10340.000000,10340,10340
unique,NaN,NaN,NaN,1179,1
top,NaN,NaN,NaN,<0.1,Estimation FAO
freq,NaN,NaN,NaN,377,10340
mean,128.480368,61113.39207,2009.944004,NaN,NaN
std,76.138781,77233.10360,5.944146,NaN,NaN
min,1.000000,21056.00000,2000.000000,NaN,NaN
25%,61.000000,21056.00000,2005.000000,NaN,NaN
50%,124.000000,21057.00000,2010.000000,NaN,NaN
75%,193.000000,21057.00000,2015.000000,NaN,NaN


In [279]:
# Recherche Valeur  .......
SecuAlim.Valeur.sort_values(ascending=True).unique() # """"array(['0.1', '0.2', '0.3', ..., '99.7', '99.9', '<0.1'], dtype=object)""""
len(SearchListe(SecuAlim,['<0.1'])) #377 sur 10340 entries

# Chamger les Valeurs '<0.1' pour passer colonne Valeur du type objet à float

SecuAlim[SecuAlim.Valeur =='<0.1'] # Selectionner les rows dont Valeur == '<0.1'

SecuAlim=SecuAlim.replace('<0.1', '0.000001')  # Remplacer '<0.1' par  '0.000001' ds le df pour garder la donnée (!= NaN ou Zéro)

# Optimisation des type du df 
# reduce_mem_usage(SecuAlim)

#DEBUG : Verif selection 
# SecuAlim[SecuAlim.Valeur =='<0.1'] #OK 
# SecuAlim.info()  # 10340 entries  pas de NaN
# SecuAlim.describe(include='all')
# SecuAlim.info(memory_usage=True)
# SearchNaN(SecuAlim ,1) # OK pas de NaN
# len(SearchListe(SecuAlim,['<0.1']))  #Nettoyage OK  
# len(SearchListe(SecuAlim,['0.000001']))  # 377 nett Ok 

#### Pivot & nettoyage NaN

In [280]:
#Jointure SecuAlim & SecuAlim_selected pour afficher le Nom des fk 
#________________________________________________________________________________________________
# SecuAlim_selected.columns   #Table pour décoder le fk
"""Index(['CodeProduit', 'Produit', 'Unité', 'Description', 'ListeProduit'], dtype='object')"""
# SecuAlim.columns 
"""Index(['CodeZone', 'CodeProduit', 'AnnéeREF', 'Valeur', 'Symbole'], dtype='object')"""
#________________________________________________________________________________________________

dtypeCat2Obj(SecuAlim) # Changer les cat en obj pour eviter les bug 
dtypeCat2Obj(SecuAlim_selected) # Changer les cat en obj pour eviter les bug 
SecuAlim=SecuAlim.merge(SecuAlim_selected.drop(columns='Description'))
ColMove(SecuAlim,'ListeProduit',1)
ColMove(SecuAlim,'CodeProduit',8)


#DEBUG
# SecuAlim.info(memory_usage=True)  #Ok Int64Index: 10340  /obj
# SearchNaN(SecuAlim ,1) # OK pas de NaN

,CodeZone,ListeProduit,AnnéeREF,Valeur,Symbole,Produit,Unité,CodeProduit
0,1,21057-Besoins énergétiques alimentaires moyens...,2001,2394,Estimation FAO,Besoins énergétiques alimentaires moyens (kcal...,Kcal/personne/jour,21057
1,1,21057-Besoins énergétiques alimentaires moyens...,2003,2418,Estimation FAO,Besoins énergétiques alimentaires moyens (kcal...,Kcal/personne/jour,21057
2,1,21057-Besoins énergétiques alimentaires moyens...,2004,2428,Estimation FAO,Besoins énergétiques alimentaires moyens (kcal...,Kcal/personne/jour,21057
3,1,21057-Besoins énergétiques alimentaires moyens...,2005,2436,Estimation FAO,Besoins énergétiques alimentaires moyens (kcal...,Kcal/personne/jour,21057
4,1,21057-Besoins énergétiques alimentaires moyens...,2006,2440,Estimation FAO,Besoins énergétiques alimentaires moyens (kcal...,Kcal/personne/jour,21057
...,...,...,...,...,...,...,...,...
10335,351,210011-Nombre de personnes sous-alimentées (mi...,2008,54.9,Estimation FAO,Nombre de personnes sous-alimentées (millions)...,millions,210011
10336,351,210011-Nombre de personnes sous-alimentées (mi...,2007,69.1,Estimation FAO,Nombre de personnes sous-alimentées (millions)...,millions,210011
10337,351,210011-Nombre de personnes sous-alimentées (mi...,2006,83.2,Estimation FAO,Nombre de personnes sous-alimentées (millions)...,millions,210011
10338,351,210011-Nombre de personnes sous-alimentées (mi...,2005,95.4,Estimation FAO,Nombre de personnes sous-alimentées (millions)...,millions,210011


In [281]:
#Creation du Pivot table pour structurer les données

SecuAlim.columns 
""" Index(['CodeZone', 'ListeProduit', 'AnnéeREF', 'Valeur', 'Symbole', 'Produit','Unité', 'CodeProduit'],  dtype='object') """

SecuAlimP=SecuAlim.pivot(index=['CodeZone','AnnéeREF'],columns=['ListeProduit'], values='Valeur').fillna(0)


# Convertion df multi-index  vers  df index simple
SecuAlimP.columns=SecuAlimP.columns.to_flat_index() # colonnes => concatener le multi index en un index simple https://datascientyst.com/flatten-multiindex-in-pandas/
SecuAlimP=SecuAlimP.reset_index() # Conversion Index en colonnes simpl
SecuAlim=SecuAlimP
dtypeCat2Obj(SecuAlim) #Dtype Category to Object (str)

SecuAlim=SecuAlim.replace(np.nan,0)

# Optimisation des type du df 
# reduce_mem_usage(SecuAlim)

# #DEBUG : Probleme NaN  et colonne Dtype Category
# SecuAlim[SecuAlim.CodeZone==351].AnnéeREF.unique()
# SecuAlim[SecuAlim.CodeZone==1].AnnéeREF.unique()
# SecuAlim.info() #PAs de NaN

# SecuAlim.info() #PAs de NaN  4074 entries,
# SearchListe(SecuAlim,[0])  # 4074 entries dont 1882 rows with [0]

### SecuAlim Geo

###### Merge SecuAlim PAYS , pop, pib 

In [282]:
# Merge SecuAlim & PAYS & pop & pib
#________________________________________________________________________________________________
# SecuAlim.columns  #4114 rows × 14 columns
"""Index(['CodeZone', 'AnnéeREF',
       '210011-Nombre de personnes sous-alimentées (millions) (moyenne sur 3 ans)',
       '21056-Besoins énergétiques alimentaires minimaux (kcal/personne/jour)',
       '21057-Besoins énergétiques alimentaires moyens (kcal/personne/jour)'],
      dtype='object', name='ListeProduit')"""  
# PAYS.columns
"""Index(['CodeContinent', 'continent', 'CodeZoneContinent', 'ZoneContinent','CodeZone', 'Zone', 'AlphaISO3'],  dtype='object')"""
# pop.columns
"""Index(['CodeZone', 'Zone', 'Année', 'Population', 'PopulationM'], dtype='object')"""
# pib.columns
"""Index(['CodeZone', 'Zone', 'Année', 'USDparHab'], dtype='object')"""
#______________________________________________________________________________________________

SecuAlimGeo=SecuAlim.rename(columns={'AnnéeREF':'Année'}).merge(PAYS,how='left').merge(pop,how='left').merge(pib,how='left').fillna(0) ##4074 rows × 14 columns

#Dtype change
for i in range(2,5):
  SecuAlimGeo.iloc[:,i]=SecuAlimGeo.iloc[:,i].astype(float)
reduce_mem_usage(SecuAlimGeo)

ColMove(SecuAlimGeo,'Zone',1)


Memory usage of dataframe is 0.38MB
Memory usage after optimization is: 0.15MB
Decreased by 60.2%


,CodeZone,Zone,Année,210011-Nombre de personnes sous-alimentées (millions) (moyenne sur 3 ans),21056-Besoins énergétiques alimentaires minimaux (kcal/personne/jour),21057-Besoins énergétiques alimentaires moyens (kcal/personne/jour),CodeContinent,continent,CodeZoneContinent,ZoneContinent,AlphaISO3,Population,PopulationM,USDparHab
0,1,Arménie,2000,0.000000,1839.0,2378.0,5300,Asie,5305,Asie occidentale,ARM,3.076000e+06,3.076172,0.0
1,1,Arménie,2001,0.799805,1849.0,2394.0,5300,Asie,5305,Asie occidentale,ARM,3.060000e+06,3.060547,0.0
2,1,Arménie,2002,0.700195,1858.0,2408.0,5300,Asie,5305,Asie occidentale,ARM,3.047000e+06,3.046875,0.0
3,1,Arménie,2003,0.600098,1865.0,2418.0,5300,Asie,5305,Asie occidentale,ARM,3.036000e+06,3.035156,0.0
4,1,Arménie,2004,0.500000,1871.0,2428.0,5300,Asie,5305,Asie occidentale,ARM,3.026000e+06,3.025391,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4069,351,Chine,2016,0.000000,1883.0,2444.0,5300,Asie,5302,Asie orientale,CHN,1.445524e+09,1446.000000,0.0
4070,351,Chine,2017,0.000000,1883.0,2444.0,5300,Asie,5302,Asie orientale,CHN,1.452626e+09,1453.000000,0.0
4071,351,Chine,2018,0.000000,1883.0,2442.0,5300,Asie,5302,Asie orientale,CHN,1.459378e+09,1459.000000,0.0
4072,351,Chine,2019,0.000000,1882.0,2440.0,5300,Asie,5302,Asie orientale,CHN,1.465634e+09,1466.000000,0.0


#### SecuAlimGeo Calculs

In [294]:
#SecuAlimGeo Calculs 
# SecuAlimGeo

# Estimation du nombre de kCal nécessaire par Pays
# Besoin Moyen  (kcal/personne/jour)	21057-Besoins énergétiques alimentaires moyens ) x PopPulation 
SecuAlimGeo['PayskCalMinAN']=SecuAlimGeo["21056-Besoins énergétiques alimentaires minimaux (kcal/personne/jour)"]*SecuAlimGeo.Population*365
SecuAlimGeo['PayskCalMoyAN']=SecuAlimGeo["21057-Besoins énergétiques alimentaires moyens (kcal/personne/jour)"]*SecuAlimGeo.Population*365
SecuAlimGeo['TauxSousAlim']=(SecuAlimGeo["210011-Nombre de personnes sous-alimentées (millions) (moyenne sur 3 ans)"]/SecuAlimGeo.PopulationM)*100
SecuAlimGeo['TauxSousAlim'].fillna(0,inplace=True) # Remplacer les Nan issus div/0 ! 
SecuAlimGeo.sort_values(by=['Année','210011-Nombre de personnes sous-alimentées (millions) (moyenne sur 3 ans)'],ascending=False, inplace=True)
SecuAlimGeo

,CodeZone,Zone,Année,210011-Nombre de personnes sous-alimentées (millions) (moyenne sur 3 ans),21056-Besoins énergétiques alimentaires minimaux (kcal/personne/jour),21057-Besoins énergétiques alimentaires moyens (kcal/personne/jour),CodeContinent,continent,CodeZoneContinent,ZoneContinent,AlphaISO3,Population,PopulationM,USDparHab,PayskCalMinAN,PayskCalMoyAN,TauxSousAlim
3086,195,Sénégal,2020,0.0,1758.0,2248.0,5100,Afrique,5105,Afrique occidentale,SEN,0.0,0.000000,0.00000,0.000000e+00,0.000000e+00,0.0
2855,178,Érythrée,2020,0.0,1735.0,2220.0,5100,Afrique,5101,Afrique orientale,ERI,0.0,0.000000,0.00000,0.000000e+00,0.000000e+00,0.0
2876,179,Qatar,2020,0.0,2056.0,2688.0,5300,Asie,5305,Asie occidentale,QAT,0.0,0.000000,0.00000,0.000000e+00,0.000000e+00,0.0
3065,194,Arabie saoudite,2020,0.0,1878.0,2428.0,5300,Asie,5305,Asie occidentale,SAU,0.0,0.000000,0.00000,0.000000e+00,0.000000e+00,0.0
2960,185,Fédération de Russie,2020,0.0,1889.0,2448.0,5400,Europe,5401,Europe orientale,RUS,0.0,0.000000,0.00000,0.000000e+00,0.000000e+00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1743,112,Jordanie,2000,0.0,1756.0,2254.0,5300,Asie,5305,Asie occidentale,JOR,4767000.0,4.765625,47.46875,3.055361e+12,3.921859e+12,0.0
1764,113,Kirghizistan,2000,0.0,1793.0,2306.0,5300,Asie,5301,Asie centrale,KGZ,4955000.0,4.953125,38.93750,3.242775e+12,4.170574e+12,0.0
1785,114,Kenya,2000,0.0,1675.0,2134.0,5100,Afrique,5101,Afrique orientale,KEN,31285000.0,31.281250,45.37500,1.912687e+13,2.436820e+13,0.0
1890,119,Lettonie,2000,0.0,1904.0,2476.0,5400,Europe,5402,Europe septentrionale,LVA,2371000.0,2.371094,50.43750,1.647750e+12,2.142768e+12,0.0


In [284]:
SecuAlimGeo.columns

Index(['CodeZone', 'Zone', 'Année',
       '210011-Nombre de personnes sous-alimentées (millions) (moyenne sur 3 ans)',
       '21056-Besoins énergétiques alimentaires minimaux (kcal/personne/jour)',
       '21057-Besoins énergétiques alimentaires moyens (kcal/personne/jour)',
       'CodeContinent', 'continent', 'CodeZoneContinent', 'ZoneContinent',
       'AlphaISO3', 'Population', 'PopulationM', 'USDparHab', 'PayskCalMinAN',
       'PayskCalMoyAN', 'TauxSousAlim'],
      dtype='object')

In [285]:
SecuAlimGeo.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4074 entries, 1516 to 4073
Data columns (total 17 columns):
 #   Column                                                                     Non-Null Count  Dtype   
---  ------                                                                     --------------  -----   
 0   CodeZone                                                                   4074 non-null   int16   
 1   Zone                                                                       4074 non-null   category
 2   Année                                                                      4074 non-null   int16   
 3   210011-Nombre de personnes sous-alimentées (millions) (moyenne sur 3 ans)  4074 non-null   float16 
 4   21056-Besoins énergétiques alimentaires minimaux (kcal/personne/jour)      4074 non-null   float16 
 5   21057-Besoins énergétiques alimentaires moyens (kcal/personne/jour)        4074 non-null   float16 
 6   CodeContinent                                

### SecuAlimMonde  2000-2019

In [293]:
SecuAlimGeoMonde=SecuAlimGeo.iloc[:, [2,3,12,14,15]].sort_values(by=['Année']).groupby(['Année']).sum()
SecuAlimGeoMonde.reset_index(inplace=True) 
SecuAlimGeoMonde.rename(columns={'210011-Nombre de personnes sous-alimentées (millions) (moyenne sur 3 ans)': 'Nb_Pers_sous-alimM','PayskCalMinAN':'kCalMinAN','PayskCalMoyAN':'kCalMoyAN'},inplace=True)
SecuAlimGeoMonde

,Année,Nb_Pers_sous-alimM,PopulationM,kCalMinAN,kCalMoyAN
0,2000,0.0,7284.0,4.828070e+15,6.230147e+15
1,2001,906.0,7364.0,4.888385e+15,6.307005e+15
2,2002,909.5,7444.0,4.951741e+15,6.390175e+15
3,2003,906.5,7524.0,5.016469e+15,6.477437e+15
4,2004,884.0,7604.0,5.080797e+15,6.561357e+15
5,2005,849.5,7688.0,5.142602e+15,6.641111e+15
6,2006,797.0,7780.0,5.209521e+15,6.730329e+15
7,2007,741.5,7864.0,5.269273e+15,6.807082e+15
8,2008,699.0,7948.0,5.327918e+15,6.885459e+15
9,2009,662.0,8032.0,5.386038e+15,6.961645e+15


In [265]:
SecuAlimGeoMonde.columns

Index(['210011-Nombre de personnes sous-alimentées (millions) (moyenne sur 3 ans)',
       'PopulationM', 'PayskCalMinAN', 'PayskCalMoyAN'],
      dtype='object')

In [264]:
df = SecuAlimGeoMonde
df = px.data.tips()
fig = px.scatter(df, x="total_bill", y="tip", color="smoker", facet_col="sex", facet_row="time")
fig.show()

ValueError: ignored

###### **<font color='red'>DEBUG Tool Search</font>**

In [164]:
#############################################
#            DEBUG Tooll Search !           #
#############################################
dt=SecuAlimGeo
dt.info()

#~~Search NaN ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
SearchNaN(dt,1)  
# SearchNaN(dt,2)
# SearchNaN(dt,2).CodeZone.sort_values(na_position='first').unique()
# len(SearchNaN(dt,2).CodeZone.sort_values(na_position='first').unique())
# SearchNaN(dt,2).Zone.sort_values(na_position='first').unique()
# SearchNaN(dt,2).Année.sort_values(na_position='first').unique()


#~~Search values ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# SearchListe(dt,[0])
# SearchListe(dt,[0]).CodeZone
# dt.CodeZone.sort_values(ascending=True,na_position='first').unique()
# dt.iloc[:,12].sort_values(ascending=True,na_position='first').unique()
#  dt.iloc[:,2].unique()

#__Liste valeur unique dans chaque colonne du df  _____________
# for col in ListeCol: 
#   # print(col,':',len(dt[col].unique()),': \n',dt[col].sort_values(ascending=True,na_position='first').unique(),'\n______________\n')
#   print(col,':',len(dt[col].unique()),': \n',dt[col].unique(),'\n______________\n') #Unsorted
#-----------

#~~ Dtype ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# dtypeCat2Obj(df)   # dtype Category => object (str)
# reduce_mem_usage(dt) # Optisation dtype
#__iloc dtype change ____________________
# for i in range(2,5):
#   dt.iloc[:,i]=dt.iloc[:,i].astype(float)
#-----------
# dt.info()



##################### Aide memoire ##################
# [i for i in range(2,12)]
####################################################

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4074 entries, 1516 to 4073
Data columns (total 17 columns):
 #   Column                                                                     Non-Null Count  Dtype   
---  ------                                                                     --------------  -----   
 0   CodeZone                                                                   4074 non-null   int16   
 1   Zone                                                                       4074 non-null   category
 2   Année                                                                      4074 non-null   int16   
 3   210011-Nombre de personnes sous-alimentées (millions) (moyenne sur 3 ans)  4074 non-null   float16 
 4   21056-Besoins énergétiques alimentaires minimaux (kcal/personne/jour)      4074 non-null   float16 
 5   21057-Besoins énergétiques alimentaires moyens (kcal/personne/jour)        4074 non-null   float16 
 6   CodeContinent                                

In [165]:
#################################  TO DELETE  ###############################################################
# Test création du code 
# temp=SecuAlim.merge(PAYS,how='left') #4074 rows × 11 columns  ok pas de NaN
# temp=SecuAlim.merge(PAYS,how='left').merge(pop,how='left') #  #4114 rows × 14 columns => 74067 rows × 14 columns   NaN pop 273 rows donnnées manquante pop  code Zone[ 13,  18,  61, 145, 177, 178, 179, 200, 201, 219, 276, 277, 299]
# # NaN pop  ['Bahreïn', 'Bhoutan', 'Guinée équatoriale','Micronésie (États fédérés de)', 'Palestine', 'Porto Rico', 'Qatar', 'Singapour', 'Somalie', 'Soudan du Sud', 'Soudan ex',  'Tonga', 'Érythrée'],
# temp=SecuAlim.merge(PAYS,how='left').merge(pop,how='left').merge(pib,how='left') ##4114 rows × 14 columns => 74067 rows × 14 columns  pop: Nan 273row
# temp=SecuAlim.merge(PAYS,how='left').merge(pop,how='left').merge(pib,how='left') ##4114 rows × 14 columns => 74067 rows × 14 columns  USD/hab : Nan  24507 => 73 pays-Année! 
# temp=SecuAlim.merge(PAYS,how='left').merge(pop,how='left').merge(pib,how='left').fillna(0) ##74067 rows × 15 columns Plus de NaN remplacé par 0! 
# temp

#########################
# DEBUG Tooll Search !  #
#########################
# dt=temp
# dt.info()
# SearchNaN(dt,1)  
# SearchNaN(dt,2)
# SearchNaN(dt,2).CodeZone.sort_values(na_position='first').unique()
# len(SearchNaN(dt,2).CodeZone.sort_values(na_position='first').unique())
# SearchNaN(dt,2).Zone.sort_values(na_position='first').unique()
# SearchNaN(dt,2).Année.sort_values(na_position='first').unique()
# SearchListe(dt,[0])

## <font color='red'>DEBUG Tool Search</font>

In [254]:
#########################
# DEBUG Tooll Search !  #
#########################
dt=SecuAlimGeo
dt.info()

#~~Search NaN ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
SearchNaN(dt,1)  
# SearchNaN(dt,2)
# SearchNaN(dt,2).CodeZone.sort_values(na_position='first').unique()
# len(SearchNaN(dt,2).CodeZone.sort_values(na_position='first').unique())
# SearchNaN(dt,2).Zone.sort_values(na_position='first').unique()
# SearchNaN(dt,2).Année.sort_values(na_position='first').unique()


#~~Search values ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# SearchListe(dt,[0])
# SearchListe(dt,[0]).CodeZone
# dt.CodeZone.sort_values(ascending=True,na_position='first').unique()
# dt.iloc[:,12].sort_values(ascending=True,na_position='first').unique()
#  dt.iloc[:,2].unique()

#__Liste valeur unique dans chaque colonne du df  _____________
# for col in ListeCol: 
#   # print(col,':',len(dt[col].unique()),': \n',dt[col].sort_values(ascending=True,na_position='first').unique(),'\n______________\n')
#   print(col,':',len(dt[col].unique()),': \n',dt[col].unique(),'\n______________\n') #Unsorted
#-----------

#~~ Dtype ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# dtypeCat2Obj(df)   # dtype Category => object (str)
# reduce_mem_usage(dt) # Optisation dtype
#__iloc dtype change ____________________
# for i in range(2,5):
#   dt.iloc[:,i]=dt.iloc[:,i].astype(float)
#-----------
# dt.info()



##################### Aide memoire ##################
# [i for i in range(2,12)]

# df.iloc[:, [2,3,12,14,15]]
####################################################

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4074 entries, 1516 to 4073
Data columns (total 17 columns):
 #   Column                                                                     Non-Null Count  Dtype   
---  ------                                                                     --------------  -----   
 0   CodeZone                                                                   4074 non-null   int16   
 1   Zone                                                                       4074 non-null   category
 2   Année                                                                      4074 non-null   int16   
 3   210011-Nombre de personnes sous-alimentées (millions) (moyenne sur 3 ans)  4074 non-null   float16 
 4   21056-Besoins énergétiques alimentaires minimaux (kcal/personne/jour)      4074 non-null   float16 
 5   21057-Besoins énergétiques alimentaires moyens (kcal/personne/jour)        4074 non-null   float16 
 6   CodeContinent                                

In [167]:
#################################  TO DELETE  ###############################################################
# Test création du code 
# temp=SecuAlim.merge(PAYS,how='left') #4074 rows × 11 columns  ok pas de NaN
# temp=SecuAlim.merge(PAYS,how='left').merge(pop,how='left') #  #4114 rows × 14 columns => 74067 rows × 14 columns   NaN pop 273 rows donnnées manquante pop  code Zone[ 13,  18,  61, 145, 177, 178, 179, 200, 201, 219, 276, 277, 299]
# # NaN pop  ['Bahreïn', 'Bhoutan', 'Guinée équatoriale','Micronésie (États fédérés de)', 'Palestine', 'Porto Rico', 'Qatar', 'Singapour', 'Somalie', 'Soudan du Sud', 'Soudan ex',  'Tonga', 'Érythrée'],
# temp=SecuAlim.merge(PAYS,how='left').merge(pop,how='left').merge(pib,how='left') ##4114 rows × 14 columns => 74067 rows × 14 columns  pop: Nan 273row
# temp=SecuAlim.merge(PAYS,how='left').merge(pop,how='left').merge(pib,how='left') ##4114 rows × 14 columns => 74067 rows × 14 columns  USD/hab : Nan  24507 => 73 pays-Année! 
# temp=SecuAlim.merge(PAYS,how='left').merge(pop,how='left').merge(pib,how='left').fillna(0) ##74067 rows × 15 columns Plus de NaN remplacé par 0! 
# temp

#########################
# DEBUG Tooll Search !  #
#########################
# dt=temp
# dt.info()
# SearchNaN(dt,1)  
# SearchNaN(dt,2)
# SearchNaN(dt,2).CodeZone.sort_values(na_position='first').unique()
# len(SearchNaN(dt,2).CodeZone.sort_values(na_position='first').unique())
# SearchNaN(dt,2).Zone.sort_values(na_position='first').unique()
# SearchNaN(dt,2).Année.sort_values(na_position='first').unique()
# SearchListe(dt,[0])

## <font color='Orange'> Aide Alimentaire 2000 - 2016 </font>

### Import et nettoyage Données FAO

In [168]:
# Aide Alimentaire (WFP) Expédition RAW dataSet FAO  source : https://www.fao.org/faostat/fr/#data/FA
AideAlimRAWPrev=pd.read_csv(DataPATH+'FAOSTAT_aideAlimentaire.csv',sep=',',usecols = [i for i in range(2,12)],nrows=5)
AideAlimRAWPrev

,Code Pays bénéficiaire (FAO),Pays bénéficiaire,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur
0,2,Afghanistan,500,L'aide alimentaire reçue,11704,Autres non-céréales,2000,2000,tonnes,55
1,2,Afghanistan,500,L'aide alimentaire reçue,11704,Autres non-céréales,2002,2002,tonnes,2313
2,2,Afghanistan,500,L'aide alimentaire reçue,11704,Autres non-céréales,2003,2003,tonnes,9821
3,2,Afghanistan,500,L'aide alimentaire reçue,11704,Autres non-céréales,2004,2004,tonnes,7332
4,2,Afghanistan,500,L'aide alimentaire reçue,11704,Autres non-céréales,2005,2005,tonnes,16985


In [169]:
# Aide Alimentaire (WFP) Expédition RAW dataSet FAO  source : https://www.fao.org/faostat/fr/#data/FA
AideAlimRAW=pd.read_csv(DataPATH+'FAOSTAT_aideAlimentaire.csv',sep=',',usecols = [i for i in range(2,12)])

reduce_mem_usage(AideAlimRAW) #Optimzation RAM usage!
# AideAlimRAW
# AideAlimRAW.info()
#AideAlimRAW.columns  #Index(['Code Pays bénéficiaire (FAO)', 'Pays bénéficiaire', 'Code Élément','Élément', 'Code Produit', 'Produit', 'Code année', 'Année', 'Unité', 'Valeur'], dtype='object')

AideAlimRAW.drop(columns=['Code année','Code Élément','Élément','Unité','Pays bénéficiaire'],inplace=True) #Suppresion colonnes inutiles
AideAlimRAW.rename(columns={'Code Pays bénéficiaire (FAO)' : 'CodeZone', 'Code Produit':'CodeProduitAA', 'Valeur':'tonnes'},inplace=True)

#DEBUG
# AideAlimRAW
# AideAlimRAW.info()
# SearchNaN(AideAlimRAW,1) #Ok Pas de NaN

# Attention CodeProduitAA  pas compatible avec CodeProduit (utilisé dans les autres fichiers de la FAO))

Memory usage of dataframe is 0.80MB
Memory usage after optimization is: 0.18MB
Decreased by 76.8%


### Données Aide Alimentaire

#### Detail : Produit par année pour Chaque Pays bénéficiaire d'Aide Alimentaire (2000 à 2016)

In [170]:
AideAlimRAW.Année.sort_values(ascending=True,na_position='first').unique()

array([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
       2011, 2012, 2013, 2014, 2015, 2016], dtype=int16)

In [171]:
# AideAlimRAW.Année.sort_values(ascending=True,na_position='first').unique()   #==> Année [2000,2016]      _
dtypeCat2Obj(geo)
dtypeCat2Obj(AideAlimRAW)
# Jointure pour data visualisation sur map 

AideAlimGeo=AideAlimRAW.merge(geo,how='left',on=['Année', 'CodeZone'])  #10423 rows × 5 columns =>Int64Index: 10443 x 13
dtypeCat2Obj(AideAlimGeo)
AideAlimGeo.info()

# DEBUG Merge 
# PAYS.query('CodeZone==206') #OK 
# geo.query('CodeZone==206') #Ok
# Com_cols=np.intersect1d(geo.columns,AideAlimRAW.columns,) # Colonnes Communes ['Année', 'CodeZone']
# diff_cols = geo.columns.difference(AideAlimRAW.columns) # Colonnes Differentes
# print(len(AideAlimRAW),len(AideAlimGeo))# Controle jointure  10423 10443

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10443 entries, 0 to 10442
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CodeZone           10443 non-null  int16  
 1   CodeProduitAA      10443 non-null  int16  
 2   Produit            10443 non-null  object 
 3   Année              10443 non-null  int16  
 4   tonnes             10443 non-null  int32  
 5   AlphaISO3          9543 non-null   object 
 6   Zone               9543 non-null   object 
 7   Population         9543 non-null   float32
 8   PopulationM        9543 non-null   float16
 9   USDparHab          9543 non-null   float16
 10  CodeContinent      9543 non-null   float64
 11  continent          9543 non-null   object 
 12  CodeZoneContinent  9543 non-null   float64
 13  ZoneContinent      9543 non-null   object 
dtypes: float16(2), float32(1), float64(2), int16(3), int32(1), object(5)
memory usage: 836.3+ KB


In [172]:
#####################   Cellul 0 surpimer ? 




# # AideAlimRAW.Année.sort_values(ascending=True,na_position='first').unique()   #==> Année [2000,2016]

# # Jointure pour data visualisation sur map 
# AideAlimGeo=AideAlimRAW.merge(geo,how='left')
# dtypeCat2Obj(AideAlimGeo)
# AideAlimGeo=AideAlimGeo.merge(PAYS,how='left')
# ColMove(AideAlimGeo,'Zone',1) # depalcer les colonnes dans le df
# AideAlimGeo

# #DEBUG 
# # print(len(AideAlimRAW),len(AideAlimGeo))# Controle jointure 
# #get_different_rows(AideAlimGeo,AideAlimRAW).CodeZone.unique()

###### Merge controle

In [173]:
# A=([ 18,  29, 357,  45, 178,  61,  76, 212, 250, 186, 196, 201, 276,206, 277])
# B=PAYS.CodeZone.unique()
# LostElement(A,B)

In [174]:
AideAlimGeo.query('CodeZone==18')

,CodeZone,CodeProduitAA,Produit,Année,tonnes,AlphaISO3,Zone,Population,PopulationM,USDparHab,CodeContinent,continent,CodeZoneContinent,ZoneContinent
968,18,11704,Autres non-céréales,2005,87,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
969,18,11704,Autres non-céréales,2006,62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
970,18,11945,Blé et Farin,2002,1970,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
971,18,12061,Céréales,2001,786,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
972,18,12061,Céréales,2002,4979,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1035,18,10031,"Riz, total",2002,3009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1036,18,10031,"Riz, total",2003,1684,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1037,18,10031,"Riz, total",2004,700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1038,18,10031,"Riz, total",2005,1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [175]:
SearchNaN(AideAlimGeo,2)
SearchNaN(AideAlimGeo,2).CodeZone.unique()
print(len(AideAlimRAW),len(AideAlimGeo)) #10423 10423

10423 10443


In [176]:
# df=AideAlimGeo
# ListeCol=df.columns
# for col in ListeCol:
#  print(col,':',len(df[col].unique()),': \n',df[col].sort_values(ascending=True,na_position='first').unique(),'\n______________\n')
# geo[geo.Année!=0].CodeZone.sort_values(ascending=True,na_position='first').unique()

In [177]:
AideAlimGeo.Population.sort_values(ascending=True,na_position='first').unique()

array([         nan, 1.390000e+05, 1.420000e+05, ..., 1.373219e+09,
       1.399454e+09, 1.430671e+09], dtype=float32)

In [178]:
##############################################################################  JOnture sur PAYS ) 
# # Jointure pour data visualisation sur map 
# AideAlim=AideAlimRAW.merge(PAYS, left_on='CodeZone',right_on='CodeZone')

# ColMove(AideAlim,'Zone',1) # depalcer les colonnes dans le df
# AideAlim
###############################################################################

#### Aide Alimentaire: Tonnages Aide Alimentaire cumulés par pays bénéficiaire (2000 à 2016)

##### Tableau: Tonnages Aide Alimentaire Annuel par pays bénéficiaire (2000 à 2016)

In [179]:
AideAlimRAW.columns #Index(['CodeZone', 'CodeProduitAA', 'Produit', 'Année', 'tonnes'], dtype='object')
# groupby 
TEMP=AideAlimRAW.groupby(by=['CodeZone','Année'])[['tonnes']].sum()
TEMP.rename(columns={"tonnes": "AideAlim_t"},inplace=True)
TEMP.reset_index(inplace=True) # convertir l'index en colonnes du df 

AideAlimGeoAN=TEMP.merge(geo).sort_values(by=["AideAlim_t"], ascending=False) # Merge plus tri Aide alim DESC
ColMove(AideAlimGeoAN,"Zone",2)
ColMove(AideAlimGeoAN,"AlphaISO3",3)
ColMove(AideAlimGeoAN,"AideAlim_t",4)

#Calcul AideAlimentaire par personne g/personne (1t=10e+6g - Pop: Millionhab)
AideAlimGeoAN["AideAlim_g/personne/jour"]=(AideAlimGeoAN.AideAlim_t/AideAlimGeoAN.PopulationM)/365
ColMove(AideAlimGeoAN,"AideAlim_g/personne/jour",6)


#DEBUG
# AideAlimGeoAN.sort_values(by=["AideAlim_g/personne/jour"], ascending=False)
# TEMP.AideAlim_t.sort_values(ascending=True,na_position='first').unique()

,CodeZone,Année,Zone,AlphaISO3,AideAlim_t,Population,AideAlim_g/personne/jour,PopulationM,USDparHab,CodeContinent,continent,CodeZoneContinent,ZoneContinent
1224,238,2003,Éthiopie,ETH,3103330,71990000.0,118.087139,72.000000,47.18750,5100,Afrique,5101,Afrique orientale
629,116,2001,République populaire démocratique de Corée,PRK,2976672,23043000.0,353.855599,23.046875,75.75000,5300,Asie,5302,Asie orientale
1221,238,2000,Éthiopie,ETH,2696066,66024000.0,111.916397,66.000000,48.21875,5100,Afrique,5101,Afrique orientale
1230,238,2009,Éthiopie,ETH,2434042,84838000.0,78.627658,84.812500,83.25000,5100,Afrique,5101,Afrique orientale
628,116,2000,République populaire démocratique de Corée,PRK,2393614,22840000.0,287.073995,22.843750,75.62500,5300,Asie,5302,Asie orientale
...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,52,2011,Azerbaïdjan,AZE,0,9146000.0,0.000000,9.148438,139.62500,5300,Asie,5305,Asie occidentale
1184,230,2000,Ukraine,UKR,0,49057000.0,0.000000,49.062500,46.59375,5400,Europe,5401,Europe orientale
1214,235,2011,Ouzbékistan,UZB,0,28977000.0,0.000000,28.984375,92.31250,5300,Asie,5301,Asie centrale
927,169,2009,Paraguay,PRY,0,6347000.0,0.000000,6.347656,83.43750,5200,Amériques,5207,Amérique du Sud


##### graphe: Tonnages Aide Alimentaire Annuel par pays bénéficiaire (2000 à 2016)

In [180]:
# AideAlimGeoAN.columns
""" Index(['CodeZone', 'Année', 'Zone', 'AlphaISO3', 'AideAlim_t', 'Population','AideAlim_g/personne/jour', 'PopulationM', 'USDparHab', 'CodeContinent','continent', 'CodeZoneContinent', 'ZoneContinent'],dtype='object')"""


" Index(['CodeZone', 'Année', 'Zone', 'AlphaISO3', 'AideAlim_t', 'Population','AideAlim_g/personne/jour', 'PopulationM', 'USDparHab', 'CodeContinent','continent', 'CodeZoneContinent', 'ZoneContinent'],dtype='object')"

In [181]:
df = AideAlimGeoAN
fig1 = px.scatter_geo(df, locations='AlphaISO3', color='continent',
                     hover_name='Zone', size='AideAlim_t',size_max=40,
                     animation_frame="Année",                  
                     title="Aide Alimentaire : Tonnes/an par Pays")
fig1.show()

In [182]:
df = AideAlimGeoAN
fig2 = px.choropleth(df, locations='AlphaISO3', color='AideAlim_g/personne/jour', color_continuous_scale='Oranges',
                     hover_name='Zone', animation_frame="Année",
                     title="Aide Alimentaire : g/hab/jour")
fig2.show()

In [183]:
# AideAlimGeoAN.columns
""" Index(['CodeZone', 'Année', 'Zone', 'AlphaISO3', 'AideAlim_t', 'Population','AideAlim_g/personne/jour', 'PopulationM', 'USDparHab', 'CodeContinent','continent', 'CodeZoneContinent', 'ZoneContinent'],dtype='object')"""


df = AideAlimGeoAN
fig3 = px.scatter_geo(df, locations='AlphaISO3', color='continent',
                     hover_name='Zone', size='AideAlim_t',size_max=40,
                     animation_frame="Année",
                     projection="natural earth",
                     title="Aide Alimentaire : Tonnes/an par Pays")

fig4 = px.choropleth(df, locations='AlphaISO3', color='AideAlim_g/personne/jour', color_continuous_scale='Oranges',
                     hover_name='Zone', 
                     animation_frame="Année",
                     title="Aide Alimentaire : g/hab/jour")

fig3.update_layout(legend=dict(yanchor="top",  y=0.99,  xanchor="left",  x=0.01))



fig3.add_trace(fig2.data[0])
for i, frame in enumerate(fig3.frames):
    fig3.frames[i].data += (fig2.frames[i].data[0],)
fig3.show()

In [184]:
df = AideAlimGeoAN

px.scatter(df,y="AideAlim_t",x='Année', size="AideAlim_t", hover_name="Zone", color="continent",size_max=50,log_x=False)

In [185]:
df = AideAlimGeoAN

px.scatter(df,y='AideAlim_g/personne/jour',x='Année', size='AideAlim_g/personne/jour', hover_name="Zone", color="continent",size_max=50,log_x=False)

##### Tableau :  Aide Alimentaire Detail : Cumul par pays Pays Année de 2000 à 2016

In [186]:
# Cumul Aide Alimentaire Année ([debut,Fin] par Pays
# AideAlimRAW.Année.sort_values(ascending=True,na_position='first').unique()  # [200,....2016]
""" [2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016], dtype=int16)"""
debut=2000
fin=2016
AideAlimPeriode=AideAlimRAW[(AideAlimRAW.Année>=debut) & (AideAlimRAW.Année<=fin)]

# groupby 
TEMP=AideAlimPeriode.groupby(by='CodeZone')[['tonnes']].sum()
TEMP.rename(columns={"tonnes": "AideAlim_t"},inplace=True)
TEMP.reset_index(inplace=True) # convertir l'index en colonnes du df 


# Merge  avec fk geo
AideAlimCumulGeo=TEMP.merge(geo[geo.Année==fin]).sort_values(by=["AideAlim_t"], ascending=False) # Merge plus tri Aide alim DESC
ColMove(AideAlimCumulGeo,"Année",12)
AideAlimCumulGeo.rename(columns={"Année": "AnnéeREFdataGeo"},inplace=True)
ColMove(AideAlimCumulGeo,"Zone",2)
ColMove(AideAlimCumulGeo,"AlphaISO3",3)
ColMove(AideAlimCumulGeo,"AideAlim_t",4)

# Renommage des colonnes Geo  dynamique
PopNew='Population_'+str(fin)
PopMNew='PopM_'+str(fin)
USDparHabNew='USDparHab_'+str(fin)
AideAlimCumulGeo.rename(columns={"Population":PopNew,"PopulationM":PopMNew,"USDparHab":USDparHabNew },inplace=True)
AideAlimCumulGeoTITRE='Cumul Aide Alimentaire par Pays de '+str(debut)+' à '+str(fin)

PercentCumul(AideAlimCumulGeo,'AideAlim_t')
AideAlimCumulGeo

,CodeZone,Zone,AlphaISO3,AideAlim_t,Population_2016,PopM_2016,USDparHab_2016,CodeContinent,continent,CodeZoneContinent,ZoneContinent,AnnéeREFdataGeo
108,238,Éthiopie,ETH,24472978,103603000.0,103.625000,106.3750,5100,Afrique,5101,Afrique orientale,2016
53,116,République populaire démocratique de Corée,PRK,18089550,25308000.0,25.312500,99.2500,5300,Asie,5302,Asie orientale,2016
93,206,Soudan,SDN,8745472,39847000.0,39.843750,103.1250,5100,Afrique,5103,Afrique septentrionale,2016
6,16,Bangladesh,BGD,7404206,157976992.0,158.000000,105.7500,5300,Asie,5303,Asie méridionale,2016
1,2,Afghanistan,AFG,6025142,35383000.0,35.375000,94.8125,5300,Asie,5303,Asie méridionale,2016
...,...,...,...,...,...,...,...,...,...,...,...,...
9,23,Belize,BLZ,296,368000.0,0.367920,102.8750,5200,Amériques,5204,Amérique centrale,2016
65,138,Mexique,MEX,200,123333000.0,123.312500,89.5000,5200,Amériques,5204,Amérique centrale,2016
78,166,Panama,PAN,50,4037000.0,4.035156,102.0000,5200,Amériques,5204,Amérique centrale,2016
79,169,Paraguay,PRY,0,6778000.0,6.777344,95.5625,5200,Amériques,5207,Amérique du Sud,2016


##### Graph :  Aide Alimentaire Detail : Cumul par pays Pays Année de 2000 à 2016

In [187]:
AideAlimCumulGeo.head(20)

,CodeZone,Zone,AlphaISO3,AideAlim_t,Population_2016,PopM_2016,USDparHab_2016,CodeContinent,continent,CodeZoneContinent,ZoneContinent,AnnéeREFdataGeo
108,238,Éthiopie,ETH,24472978,103603000.0,103.625000,106.3750,5100,Afrique,5101,Afrique orientale,2016
53,116,République populaire démocratique de Corée,PRK,18089550,25308000.0,25.312500,99.2500,5300,Asie,5302,Asie orientale,2016
93,206,Soudan,SDN,8745472,39847000.0,39.843750,103.1250,5100,Afrique,5103,Afrique septentrionale,2016
6,16,Bangladesh,BGD,7404206,157976992.0,158.000000,105.7500,5300,Asie,5303,Asie méridionale,2016
1,2,Afghanistan,AFG,6025142,35383000.0,35.375000,94.8125,5300,Asie,5303,Asie méridionale,2016
51,114,Kenya,KEN,5662476,49052000.0,49.062500,102.0625,5100,Afrique,5101,Afrique orientale,2016
68,144,Mozambique,MOZ,3797188,27830000.0,27.828125,72.0625,5100,Afrique,5101,Afrique orientale,2016
39,93,Haïti,HTI,3776454,10840000.0,10.843750,89.6250,5200,Amériques,5206,Caraïbes,2016
84,181,Zimbabwe,ZWE,3725786,14030000.0,14.031250,102.1875,5100,Afrique,5101,Afrique orientale,2016
109,249,Yémen,YEM,3252618,27168000.0,27.171875,96.8125,5300,Asie,5305,Asie occidentale,2016


In [188]:
import plotly.express as px
df = AideAlimCumulGeo
fig = px.scatter_geo(df, locations="AlphaISO3", color="continent",
                     hover_name="Zone", size="AideAlim_t",size_max=30,
                     projection="natural earth")
fig.show()

In [189]:
df = AideAlimCumulGeo
fig = px.strip(df, x="AideAlim_t", hover_data=['Zone'], color="continent")
fig.show()

In [190]:
df = AideAlimCumulGeo
fig = px.histogram(df, x="AideAlim_t",y="Population_2016", hover_name="Zone", color="continent",marginal="rug")
fig.show()

In [191]:
df = AideAlimCumulGeo
fig = px.bar(df, x="AideAlim_t",y="continent", hover_name="Zone")
fig.show()

In [192]:
df = AideAlimCumulGeo
fig = px.bar(df, x="AideAlim_t",y="Zone", hover_name="Zone")
fig.show()

In [222]:
df = AideAlimCumulGeo
fig = px.sunburst(df, values="AideAlim_t",path=["continent","Zone"], hover_name="Zone")
fig.show()

In [194]:
df = AideAlimCumulGeo
fig = px.treemap(df,values="AideAlim_t",path=["continent","Zone"], hover_name="Zone")
# fig = px.treemap(df, color="Population_2016",values="AideAlim_t",path=["continent","Zone"], hover_name="Zone")
# fig = px.treemap(df, values="AideAlim_t",path=["continent","ZoneContinent","Zone"], hover_name="Zone")
fig.show()

In [195]:
df = AideAlimCumulGeo
fig = px.choropleth(df,color="AideAlim_t", hover_name="Zone",locations="AlphaISO3")
# fig = px.treemap(df, color="Population_2016",values="AideAlim_t",path=["continent","Zone"], hover_name="Zone")
# fig = px.treemap(df, values="AideAlim_t",path=["continent","ZoneContinent","Zone"], hover_name="Zone")
fig.show()

In [196]:
px.scatter(df,y="AideAlim_t",x="PopM_2016", size="AideAlim_t", hover_name="Zone", color="continent",size_max=50,log_x=True)

In [197]:
# figTEST=px.scatter(df,y="AideAlim_t",x="PopM_2016", size="AideAlim_t", hover_name="Zone", color="continent",size_max=50,log_x=True)
# figTEST.show("json")

In [198]:
AideAlimCumulGeo

,CodeZone,Zone,AlphaISO3,AideAlim_t,Population_2016,PopM_2016,USDparHab_2016,CodeContinent,continent,CodeZoneContinent,ZoneContinent,AnnéeREFdataGeo
108,238,Éthiopie,ETH,24472978,103603000.0,103.625000,106.3750,5100,Afrique,5101,Afrique orientale,2016
53,116,République populaire démocratique de Corée,PRK,18089550,25308000.0,25.312500,99.2500,5300,Asie,5302,Asie orientale,2016
93,206,Soudan,SDN,8745472,39847000.0,39.843750,103.1250,5100,Afrique,5103,Afrique septentrionale,2016
6,16,Bangladesh,BGD,7404206,157976992.0,158.000000,105.7500,5300,Asie,5303,Asie méridionale,2016
1,2,Afghanistan,AFG,6025142,35383000.0,35.375000,94.8125,5300,Asie,5303,Asie méridionale,2016
...,...,...,...,...,...,...,...,...,...,...,...,...
9,23,Belize,BLZ,296,368000.0,0.367920,102.8750,5200,Amériques,5204,Amérique centrale,2016
65,138,Mexique,MEX,200,123333000.0,123.312500,89.5000,5200,Amériques,5204,Amérique centrale,2016
78,166,Panama,PAN,50,4037000.0,4.035156,102.0000,5200,Amériques,5204,Amérique centrale,2016
79,169,Paraguay,PRY,0,6778000.0,6.777344,95.5625,5200,Amériques,5207,Amérique du Sud,2016


## <font color='Orange'> Bilan Alimentaire 2010-2020 </Font>

### Import data FAO

In [199]:
#Bilan Alimentaire 2010-2020 
#Source https://www.fao.org/faostat/fr/#data/FBS  Données completes 
# Les fichiers ont été normalisés et reduits aux clefs  pour optimiser le chargement (utilisation Power Query dans Excel )

import numpy as np
import pandas as pd
DataPATH='https://raw.githubusercontent.com/vincent-arese/OC-P04VA/main/FAOdata/'


# Chargment des fichiers 
bilan=pd.read_csv(DataPATH+'FAOBilanAlimentaireBilanUTF8.zip',sep=';',usecols = [i for i in range(5)])
dispo=pd.read_csv(DataPATH+'FAOBilanAlimentaireDispoUTF8.zip',sep=';',usecols = [i for i in range(5)])
CodeElement=pd.read_csv(DataPATH+'FAOBilanAlimentaireCodeUTF8.csv',sep=';')
CodeProduitBA=pd.read_csv(DataPATH+'DicoProduitBilanAlimFAO.csv',sep=';',usecols = [i for i in range(8)])
 ######## CODE EN TEST ####################################################################################

# Consulter Liste Produit FOA  fichier Excel : https://github.com/vincent-arese/OC-P04VA/blob/main/FAOdata/DicoProduitBilanAlimFAO.xlsx?raw=true


# reduce_mem_usage(bilan) #Optimzation RAM usage! 38.44MB  to 11.53MB
# reduce_mem_usage(dispo) #Optimzation RAM usage! 32.79MB to 9.84MB
# reduce_mem_usage(CodeElement) #Optimzation RAM usage! 
reduce_mem_usage(CodeProduitBA) #Optimzation RAM usage!  #CodeProduitBA.info()
# reduce_mem_usage(ListeProduitsFAO) #Optimzation RAM usage!

Memory usage of dataframe is 0.01MB
Memory usage after optimization is: 0.01MB
Decreased by -22.2%


,CodeProduit,Produit,CodeOrigine,Origine,CodeCatProduits,GroupeProduits,Coefficient,Description
0,2511,Blé et produits,2903,Produits Vegetaux,2905,Céréales - Excl Bière,1,"Composition par défaut: 15 Blé, 16 Farine de b..."
1,2513,Orge et produits,2903,Produits Vegetaux,2905,Céréales - Excl Bière,1,"Composition par défaut: 44 Orge, 45 Gruau d'or..."
2,2514,Maïs et produits,2903,Produits Vegetaux,2905,Céréales - Excl Bière,1,"Composition par défaut: 56 Maïs, 57 Germe de m..."
3,2515,Seigle et produits,2903,Produits Vegetaux,2905,Céréales - Excl Bière,1,"Composition par défaut: 71 Seigle, 72 Farine d..."
4,2516,Avoine,2903,Produits Vegetaux,2905,Céréales - Excl Bière,1,"Composition par défaut: 75 Avoine, 76 Avoine l..."
...,...,...,...,...,...,...,...,...
93,2781,Huiles de Poissons,2941,Produits Animaux,2946,Graisses animales,1,"Composition par défaut: 1509 HuiPoiEaDou, 1522..."
94,2782,Huiles de Foie de Poisso,2941,Produits Animaux,2946,Graisses animales,1,"Composition par défaut: 1510 HuFoPoiEaDou, 152..."
95,2807,Riz et produits,2903,Produits Vegetaux,2905,Céréales - Excl Bière,1,"Composition par défaut: 27 Riz, paddy, 28 Riz,..."
96,2848,Lait - Excl Beurre,2941,Produits Animaux,2948,Lait - Excl Beurre,1,"Composition par défaut: 882 Lait, entier frais..."


### Nettoyage et formatage

##### Nettoyage & Jointure

In [200]:
# Nettoyage
# CodeZone:  Drop  CodeZone correspondant à des agrégats ( continent, groupe de pays ........... ) =>  CodeZone>=400 ! 
''' 'Pays les moins développés' 'Pays sans littoral en voie de développement' 'Petites îles en voie de Développement'  'Pays à faible revenu et à déficit vivrier' 'Pays à faible revenu'  'Pays à revenu moyen' 'High-income economies'  'Upper-middle-income economies' '''
# bilan.columns #Index(['CodeZone', 'CodeProduit', 'CodeElément', 'Année', 'Valeur'], dtype='object')
# dispo.CodeZone.unique()
# bilan.CodeZone.unique()


#CodeProduit: Drop CodeProduits correspondant à des agrégats ( Produits Animaux/Végétaux  groupe produits (Céréales - Excl Bière, Cultures Sucrieres, légumes ,  Fruits, Epices, Poisson & Fruits de Mer )........... ) =>  CodeProduits>=2900 ! 
# testx=bilan[(bilan.CodeZone>= 400) & (bilan.CodeProduit>=2900)]
# testx.CodeProduit.unique()
# cf DicoProduitBilamAlimFAO : https://github.com/vincent-arese/OC-P04VA/blob/main/FAOdata/dico/DicoProduitBilanAlimFAO.xlsx?raw=true

# Restriction bilan & dispo
bilan.drop(bilan.loc[(bilan.CodeZone>= 400) | (bilan.CodeProduit>=2900)].index, inplace=True)
dispo.drop(dispo.loc[(dispo.CodeZone>= 400) | (dispo.CodeProduit>=2900)].index, inplace=True)


#Jointure pour créer le dataframe BilanAlim
BilanAlim=pd.merge(bilan,dispo,how='outer')
BilanAlim=BilanAlim.merge(CodeElement,how='left',on='CodeElément')
BilanAlim.Elément=BilanAlim.CodeElément.astype(str)+'-'+BilanAlim.Elément
BilanAlim.drop(columns=['CodeElément'],inplace=True)

reduce_mem_usage(BilanAlim) #Optimzation RAM usage!



#DEBUG  NaN  sur les codes produits > 20900  (Catégories produits agrégées (Produits Végétaux/Animaux, Viande, legumes .......))
# x=BilanAlim.CodeProduit.sort_values(ascending=True).unique()
# y=CodeProduitBA.CodeProduit.sort_values(ascending=True).unique()
# type(z)
# print(len(y),len(x))
# LostElement(x,y)
# #BilanAlim.drop(BilanAlim[BilanAlim.CodeProduit>=2900].index,inplace=True)


# BilanAlim.head(1)
#DEBUG
# BilanAlim.info()
# BilanAlim.Année.unique() # [2010,2019]
# BilanAlim.columns # Index(['CodeZone', 'CodeProduit', 'CodeElément', 'Année', 'Valeur', 'Elément','Unité'],  dtype='object')
# SearchNaN(BilanAlim,1)  # OK pas de NAN 1067499 rows × 6 columns
# SearchListe(BilanAlim,[0]) # Aucun [0]

Memory usage of dataframe is 57.01MB
Memory usage after optimization is: 20.36MB
Decreased by 64.3%


,CodeZone,CodeProduit,Année,Valeur,Elément,Unité
0,2,2511,2010,4532.00,5511-Production,Milliers de tonnes
1,2,2511,2011,3388.00,5511-Production,Milliers de tonnes
2,2,2511,2012,5050.00,5511-Production,Milliers de tonnes
3,2,2511,2013,5169.00,5511-Production,Milliers de tonnes
4,2,2511,2014,5370.00,5511-Production,Milliers de tonnes
...,...,...,...,...,...,...
1067494,181,2899,2015,0.03,684-Disponibilité de matière grasse en quantit...,g/personne/jour
1067495,181,2899,2016,0.03,684-Disponibilité de matière grasse en quantit...,g/personne/jour
1067496,181,2899,2017,0.01,684-Disponibilité de matière grasse en quantit...,g/personne/jour
1067497,181,2899,2018,0.01,684-Disponibilité de matière grasse en quantit...,g/personne/jour


In [201]:
#Creation du Pivot table pour structurer les données
BilanAlim=BilanAlim.pivot(index=['CodeZone','CodeProduit','Année'],columns=['Elément'], values='Valeur').fillna(0)  # Remplacer le NaN (Colonne vide) par 0
BilanAlim   # 1067499 rows × 6 columns =>142669 rows × 17 columns

# Convertion df multi-index  vers  df index simple
BilanAlim.columns=BilanAlim.columns.to_flat_index() # colonnes => concatener le multi index en un index simple https://datascientyst.com/flatten-multiindex-in-pandas/
BilanAlim=BilanAlim.reset_index() # Conversion Index en colonnes simple.
reduce_mem_usage(BilanAlim) #Optimzation RAM usage!   142669 rows × 20 columns


###DEBUG
# SearchNaN(BilanAlim,1)# Ok pas de Nan / 142669 rows × 20 columns


Memory usage of dataframe is 12.52MB
Memory usage after optimization is: 7.89MB
Decreased by 37.0%


Elément,CodeZone,CodeProduit,Année,5072-Variation de stock,5123-Pertes,5131-Traitement,5142-Nourriture,5154-Autres utilisations (non alimentaire),5170-Résidus,5171-Alimentation pour touristes,5301-Disponibilité intérieure,5511-Production,5521-Aliments pour animaux,5527-Semences,5611-Importations - Quantité,5911-Exportations - Quantité,645-Disponibilité alimentaire en quantité (kg/personne/an),664-Disponibilité alimentaire (Kcal/personne/jour),674-Disponibilité de protéines en quantité (g/personne/jour),684-Disponibilité de matière grasse en quantité (g/personne/jour)
0,1,2511,2010,-43.0,8.0,16.0,370.0,0.0,100.0,0.0,577.0,183.0,61.0,22.0,351.0,0.0,128.625000,1026.0,30.656250,3.660156
1,1,2511,2011,21.0,9.0,16.0,384.0,0.0,88.0,0.0,581.0,224.0,65.0,20.0,378.0,0.0,133.625000,1061.0,31.656250,3.759766
2,1,2511,2012,0.0,11.0,17.0,367.0,0.0,222.0,0.0,733.0,243.0,92.0,23.0,490.0,1.0,127.312500,1013.0,30.156250,3.580078
3,1,2511,2013,30.0,16.0,17.0,379.0,0.0,101.0,0.0,636.0,312.0,93.0,30.0,355.0,1.0,130.750000,1040.0,30.984375,3.640625
4,1,2511,2014,42.0,30.0,16.0,406.0,0.0,46.0,0.0,639.0,338.0,110.0,32.0,344.0,1.0,139.375000,1107.0,33.000000,3.900391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142664,351,2899,2015,28.0,0.0,0.0,230.0,0.0,-1356.0,6.0,-1120.0,0.0,0.0,0.0,749.0,1842.0,0.160034,0.0,0.010002,0.000000
142665,351,2899,2016,55.0,0.0,0.0,218.0,0.0,-1335.0,7.0,-1110.0,0.0,0.0,0.0,812.0,1867.0,0.150024,0.0,0.000000,0.000000
142666,351,2899,2017,60.0,0.0,0.0,207.0,0.0,-1481.0,7.0,-1267.0,0.0,0.0,0.0,890.0,2097.0,0.140015,0.0,0.000000,0.000000
142667,351,2899,2018,42.0,0.0,0.0,202.0,0.0,-1562.0,9.0,-1351.0,0.0,0.0,0.0,965.0,2274.0,0.140015,0.0,0.000000,0.000000


##### Merge pour faire apparaitre les fk

In [202]:
# Merge pour faire apparaitre le texte correspondant aux fk Code.....

#Detail des df  : 
#BilanAlim.columns
""" Index(['CodeZone', 'CodeProduit', 'Année', '5072-Variation de stock', '5123-Pertes', '5131-Traitement',...], dtype='object', name='Elément')"""
# CodeProduitBA.columns 
"""Index(['CodeProduit', 'Produit', 'CodeOrigine', 'Origine', 'CodeCatProduits','GroupeProduits', 'Coefficient', 'Description'], dtype='object')"""

codePBA=CodeProduitBA.iloc[:,[i for i in range(6)]]  # selection des 6 premiere colonnes  (tauf sauf 'Coefficient', 'Description' )

############################################### Merge sans le Geo #######################################################
BilanAlim=BilanAlim.merge(codePBA,how='left',on='CodeProduit') #  Ok 142669 rows × 20 columns => 142669 rows × 25 columns
BilanAlim
############################################################################################################################



#  # Suprimer les dtype Catergory  
# dtypeCat2Obj(geo)
# dtypeCat2Obj(codePBA)
# dtypeCat2Obj(BilanAlim)


# # # ################################################################################################# Ancien code depalce pares calcul 

# # BilanAlimGeo=BilanAlim.merge(codePBA,how='left',on='CodeProduit').merge(geo, how='left')   

# # ColMove(BilanAlimGeo,'Zone',1)
# # ColMove(BilanAlimGeo,'Produit',2)
# # BilanAlimGeo.info()
# BilanAlimGeo  #144590 rows × 34 columns
# # # ################################################################################

 #DEBUG 
# # temp=BilanAlim.merge(codePBA,how='left',on='CodeProduit')    #Ok pas de NAN
# # temp=BilanAlim.merge(codePBA,how='left',on='CodeProduit').merge(geo)
# BilanAlim.info(memory_usage=True)
# SearchNaN(BilanAlim,1) # Ok Pas de NaN



,CodeZone,CodeProduit,Année,5072-Variation de stock,5123-Pertes,5131-Traitement,5142-Nourriture,5154-Autres utilisations (non alimentaire),5170-Résidus,5171-Alimentation pour touristes,5301-Disponibilité intérieure,5511-Production,5521-Aliments pour animaux,5527-Semences,5611-Importations - Quantité,5911-Exportations - Quantité,645-Disponibilité alimentaire en quantité (kg/personne/an),664-Disponibilité alimentaire (Kcal/personne/jour),674-Disponibilité de protéines en quantité (g/personne/jour),684-Disponibilité de matière grasse en quantité (g/personne/jour),Produit,CodeOrigine,Origine,CodeCatProduits,GroupeProduits
0,1,2511,2010,-43.0,8.0,16.0,370.0,0.0,100.0,0.0,577.0,183.0,61.0,22.0,351.0,0.0,128.625000,1026.0,30.656250,3.660156,Blé et produits,2903,Produits Vegetaux,2905,Céréales - Excl Bière
1,1,2511,2011,21.0,9.0,16.0,384.0,0.0,88.0,0.0,581.0,224.0,65.0,20.0,378.0,0.0,133.625000,1061.0,31.656250,3.759766,Blé et produits,2903,Produits Vegetaux,2905,Céréales - Excl Bière
2,1,2511,2012,0.0,11.0,17.0,367.0,0.0,222.0,0.0,733.0,243.0,92.0,23.0,490.0,1.0,127.312500,1013.0,30.156250,3.580078,Blé et produits,2903,Produits Vegetaux,2905,Céréales - Excl Bière
3,1,2511,2013,30.0,16.0,17.0,379.0,0.0,101.0,0.0,636.0,312.0,93.0,30.0,355.0,1.0,130.750000,1040.0,30.984375,3.640625,Blé et produits,2903,Produits Vegetaux,2905,Céréales - Excl Bière
4,1,2511,2014,42.0,30.0,16.0,406.0,0.0,46.0,0.0,639.0,338.0,110.0,32.0,344.0,1.0,139.375000,1107.0,33.000000,3.900391,Blé et produits,2903,Produits Vegetaux,2905,Céréales - Excl Bière
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142664,351,2899,2015,28.0,0.0,0.0,230.0,0.0,-1356.0,6.0,-1120.0,0.0,0.0,0.0,749.0,1842.0,0.160034,0.0,0.010002,0.000000,Miscellanees,2903,Produits Vegetaux,2928,Miscellanees
142665,351,2899,2016,55.0,0.0,0.0,218.0,0.0,-1335.0,7.0,-1110.0,0.0,0.0,0.0,812.0,1867.0,0.150024,0.0,0.000000,0.000000,Miscellanees,2903,Produits Vegetaux,2928,Miscellanees
142666,351,2899,2017,60.0,0.0,0.0,207.0,0.0,-1481.0,7.0,-1267.0,0.0,0.0,0.0,890.0,2097.0,0.140015,0.0,0.000000,0.000000,Miscellanees,2903,Produits Vegetaux,2928,Miscellanees
142667,351,2899,2018,42.0,0.0,0.0,202.0,0.0,-1562.0,9.0,-1351.0,0.0,0.0,0.0,965.0,2274.0,0.140015,0.0,0.000000,0.000000,Miscellanees,2903,Produits Vegetaux,2928,Miscellanees


####  Bilan ALim Calcul

In [203]:
BilanAlim

,CodeZone,CodeProduit,Année,5072-Variation de stock,5123-Pertes,5131-Traitement,5142-Nourriture,5154-Autres utilisations (non alimentaire),5170-Résidus,5171-Alimentation pour touristes,5301-Disponibilité intérieure,5511-Production,5521-Aliments pour animaux,5527-Semences,5611-Importations - Quantité,5911-Exportations - Quantité,645-Disponibilité alimentaire en quantité (kg/personne/an),664-Disponibilité alimentaire (Kcal/personne/jour),674-Disponibilité de protéines en quantité (g/personne/jour),684-Disponibilité de matière grasse en quantité (g/personne/jour),Produit,CodeOrigine,Origine,CodeCatProduits,GroupeProduits
0,1,2511,2010,-43.0,8.0,16.0,370.0,0.0,100.0,0.0,577.0,183.0,61.0,22.0,351.0,0.0,128.625000,1026.0,30.656250,3.660156,Blé et produits,2903,Produits Vegetaux,2905,Céréales - Excl Bière
1,1,2511,2011,21.0,9.0,16.0,384.0,0.0,88.0,0.0,581.0,224.0,65.0,20.0,378.0,0.0,133.625000,1061.0,31.656250,3.759766,Blé et produits,2903,Produits Vegetaux,2905,Céréales - Excl Bière
2,1,2511,2012,0.0,11.0,17.0,367.0,0.0,222.0,0.0,733.0,243.0,92.0,23.0,490.0,1.0,127.312500,1013.0,30.156250,3.580078,Blé et produits,2903,Produits Vegetaux,2905,Céréales - Excl Bière
3,1,2511,2013,30.0,16.0,17.0,379.0,0.0,101.0,0.0,636.0,312.0,93.0,30.0,355.0,1.0,130.750000,1040.0,30.984375,3.640625,Blé et produits,2903,Produits Vegetaux,2905,Céréales - Excl Bière
4,1,2511,2014,42.0,30.0,16.0,406.0,0.0,46.0,0.0,639.0,338.0,110.0,32.0,344.0,1.0,139.375000,1107.0,33.000000,3.900391,Blé et produits,2903,Produits Vegetaux,2905,Céréales - Excl Bière
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142664,351,2899,2015,28.0,0.0,0.0,230.0,0.0,-1356.0,6.0,-1120.0,0.0,0.0,0.0,749.0,1842.0,0.160034,0.0,0.010002,0.000000,Miscellanees,2903,Produits Vegetaux,2928,Miscellanees
142665,351,2899,2016,55.0,0.0,0.0,218.0,0.0,-1335.0,7.0,-1110.0,0.0,0.0,0.0,812.0,1867.0,0.150024,0.0,0.000000,0.000000,Miscellanees,2903,Produits Vegetaux,2928,Miscellanees
142666,351,2899,2017,60.0,0.0,0.0,207.0,0.0,-1481.0,7.0,-1267.0,0.0,0.0,0.0,890.0,2097.0,0.140015,0.0,0.000000,0.000000,Miscellanees,2903,Produits Vegetaux,2928,Miscellanees
142667,351,2899,2018,42.0,0.0,0.0,202.0,0.0,-1562.0,9.0,-1351.0,0.0,0.0,0.0,965.0,2274.0,0.140015,0.0,0.000000,0.000000,Miscellanees,2903,Produits Vegetaux,2928,Miscellanees


In [204]:
# BilanAlim.columns
""" Index(['CodeZone', 'CodeProduit', 'Année', '5072-Variation de stock',  '5123-Pertes', '5131-Traitement', '5142-Nourriture', , '5170-Résidus', '5171-Alimentation pour touristes', '5301-Disponibilité intérieure', '5511-Production', '5521-Aliments pour animaux', '5527-Semences',
       '5611-Importations - Quantité', '5911-Exportations - Quantité', '645-Disponibilité alimentaire en quantité (kg/personne/an)',   '664-Disponibilité alimentaire (Kcal/personne/jour)',    '674-Disponibilité de protéines en quantité (g/personne/jour)',  '684-Disponibilité de matière grasse en quantité (g/personne/jour)'],
      dtype='object', name='Elément')"""

# 5142-Nourriture' Milliers de tonnes
BilanAlim['%Nouriture']=(BilanAlim['5142-Nourriture']/BilanAlim['5301-Disponibilité intérieure'])*100
BilanAlim['%Pertes']=(BilanAlim['5123-Pertes']/BilanAlim['5301-Disponibilité intérieure'])*100
BilanAlim['%AlimAnimale']=(BilanAlim['5521-Aliments pour animaux']/BilanAlim['5301-Disponibilité intérieure'])*100
BilanAlim['%NonAlim']=(BilanAlim['5154-Autres utilisations (non alimentaire)']/BilanAlim['5301-Disponibilité intérieure'])*100
BilanAlim['%NAutres']=100-(BilanAlim['%Nouriture']+BilanAlim['%Pertes']+BilanAlim['%AlimAnimale']+BilanAlim['%NonAlim'])
BilanAlim

,CodeZone,CodeProduit,Année,5072-Variation de stock,5123-Pertes,5131-Traitement,5142-Nourriture,5154-Autres utilisations (non alimentaire),5170-Résidus,5171-Alimentation pour touristes,5301-Disponibilité intérieure,5511-Production,5521-Aliments pour animaux,5527-Semences,5611-Importations - Quantité,5911-Exportations - Quantité,645-Disponibilité alimentaire en quantité (kg/personne/an),664-Disponibilité alimentaire (Kcal/personne/jour),674-Disponibilité de protéines en quantité (g/personne/jour),684-Disponibilité de matière grasse en quantité (g/personne/jour),Produit,CodeOrigine,Origine,CodeCatProduits,GroupeProduits,%Nouriture,%Pertes,%AlimAnimale,%NonAlim,%NAutres
0,1,2511,2010,-43.0,8.0,16.0,370.0,0.0,100.0,0.0,577.0,183.0,61.0,22.0,351.0,0.0,128.625000,1026.0,30.656250,3.660156,Blé et produits,2903,Produits Vegetaux,2905,Céréales - Excl Bière,64.124779,1.386482,10.571924,0.0,23.916817
1,1,2511,2011,21.0,9.0,16.0,384.0,0.0,88.0,0.0,581.0,224.0,65.0,20.0,378.0,0.0,133.625000,1061.0,31.656250,3.759766,Blé et produits,2903,Produits Vegetaux,2905,Céréales - Excl Bière,66.092941,1.549053,11.187608,0.0,21.170395
2,1,2511,2012,0.0,11.0,17.0,367.0,0.0,222.0,0.0,733.0,243.0,92.0,23.0,490.0,1.0,127.312500,1013.0,30.156250,3.580078,Blé et produits,2903,Produits Vegetaux,2905,Céréales - Excl Bière,50.068211,1.500682,12.551160,0.0,35.879944
3,1,2511,2013,30.0,16.0,17.0,379.0,0.0,101.0,0.0,636.0,312.0,93.0,30.0,355.0,1.0,130.750000,1040.0,30.984375,3.640625,Blé et produits,2903,Produits Vegetaux,2905,Céréales - Excl Bière,59.591190,2.515723,14.622643,0.0,23.270447
4,1,2511,2014,42.0,30.0,16.0,406.0,0.0,46.0,0.0,639.0,338.0,110.0,32.0,344.0,1.0,139.375000,1107.0,33.000000,3.900391,Blé et produits,2903,Produits Vegetaux,2905,Céréales - Excl Bière,63.536774,4.694836,17.214397,0.0,14.553986
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142664,351,2899,2015,28.0,0.0,0.0,230.0,0.0,-1356.0,6.0,-1120.0,0.0,0.0,0.0,749.0,1842.0,0.160034,0.0,0.010002,0.000000,Miscellanees,2903,Produits Vegetaux,2928,Miscellanees,-20.535715,-0.000000,-0.000000,-0.0,120.535713
142665,351,2899,2016,55.0,0.0,0.0,218.0,0.0,-1335.0,7.0,-1110.0,0.0,0.0,0.0,812.0,1867.0,0.150024,0.0,0.000000,0.000000,Miscellanees,2903,Produits Vegetaux,2928,Miscellanees,-19.639639,-0.000000,-0.000000,-0.0,119.639641
142666,351,2899,2017,60.0,0.0,0.0,207.0,0.0,-1481.0,7.0,-1267.0,0.0,0.0,0.0,890.0,2097.0,0.140015,0.0,0.000000,0.000000,Miscellanees,2903,Produits Vegetaux,2928,Miscellanees,-16.337807,-0.000000,-0.000000,-0.0,116.337807
142667,351,2899,2018,42.0,0.0,0.0,202.0,0.0,-1562.0,9.0,-1351.0,0.0,0.0,0.0,965.0,2274.0,0.140015,0.0,0.000000,0.000000,Miscellanees,2903,Produits Vegetaux,2928,Miscellanees,-14.951888,-0.000000,-0.000000,-0.0,114.951889


#### Bilan Alim Geo DEBUG 

In [205]:
geo

,CodeZone,AlphaISO3,Zone,Année,Population,PopulationM,USDparHab,CodeContinent,continent,CodeZoneContinent,ZoneContinent
0,1,ARM,Arménie,2000,3.076000e+06,3.076172,52.28125,5300,Asie,5305,Asie occidentale
1,1,ARM,Arménie,2001,3.060000e+06,3.060547,52.90625,5300,Asie,5305,Asie occidentale
2,1,ARM,Arménie,2002,3.047000e+06,3.046875,51.56250,5300,Asie,5305,Asie occidentale
3,1,ARM,Arménie,2003,3.036000e+06,3.035156,53.43750,5300,Asie,5305,Asie occidentale
4,1,ARM,Arménie,2004,3.026000e+06,3.025391,61.62500,5300,Asie,5305,Asie occidentale
...,...,...,...,...,...,...,...,...,...,...,...
3724,357,CHN,Chine (exc. Hong-Kong et Macao),2017,1.421022e+09,1421.000000,97.37500,5300,Asie,5302,Asie orientale
3725,357,CHN,Chine (exc. Hong-Kong et Macao),2018,1.459378e+09,1459.000000,103.00000,5300,Asie,5302,Asie orientale
3726,357,CHN,Chine (exc. Hong-Kong et Macao),2018,1.427648e+09,1428.000000,103.00000,5300,Asie,5302,Asie orientale
3727,357,CHN,Chine (exc. Hong-Kong et Macao),2019,1.465634e+09,1466.000000,100.18750,5300,Asie,5302,Asie orientale


In [206]:
BilanAlim

,CodeZone,CodeProduit,Année,5072-Variation de stock,5123-Pertes,5131-Traitement,5142-Nourriture,5154-Autres utilisations (non alimentaire),5170-Résidus,5171-Alimentation pour touristes,5301-Disponibilité intérieure,5511-Production,5521-Aliments pour animaux,5527-Semences,5611-Importations - Quantité,5911-Exportations - Quantité,645-Disponibilité alimentaire en quantité (kg/personne/an),664-Disponibilité alimentaire (Kcal/personne/jour),674-Disponibilité de protéines en quantité (g/personne/jour),684-Disponibilité de matière grasse en quantité (g/personne/jour),Produit,CodeOrigine,Origine,CodeCatProduits,GroupeProduits,%Nouriture,%Pertes,%AlimAnimale,%NonAlim,%NAutres
0,1,2511,2010,-43.0,8.0,16.0,370.0,0.0,100.0,0.0,577.0,183.0,61.0,22.0,351.0,0.0,128.625000,1026.0,30.656250,3.660156,Blé et produits,2903,Produits Vegetaux,2905,Céréales - Excl Bière,64.124779,1.386482,10.571924,0.0,23.916817
1,1,2511,2011,21.0,9.0,16.0,384.0,0.0,88.0,0.0,581.0,224.0,65.0,20.0,378.0,0.0,133.625000,1061.0,31.656250,3.759766,Blé et produits,2903,Produits Vegetaux,2905,Céréales - Excl Bière,66.092941,1.549053,11.187608,0.0,21.170395
2,1,2511,2012,0.0,11.0,17.0,367.0,0.0,222.0,0.0,733.0,243.0,92.0,23.0,490.0,1.0,127.312500,1013.0,30.156250,3.580078,Blé et produits,2903,Produits Vegetaux,2905,Céréales - Excl Bière,50.068211,1.500682,12.551160,0.0,35.879944
3,1,2511,2013,30.0,16.0,17.0,379.0,0.0,101.0,0.0,636.0,312.0,93.0,30.0,355.0,1.0,130.750000,1040.0,30.984375,3.640625,Blé et produits,2903,Produits Vegetaux,2905,Céréales - Excl Bière,59.591190,2.515723,14.622643,0.0,23.270447
4,1,2511,2014,42.0,30.0,16.0,406.0,0.0,46.0,0.0,639.0,338.0,110.0,32.0,344.0,1.0,139.375000,1107.0,33.000000,3.900391,Blé et produits,2903,Produits Vegetaux,2905,Céréales - Excl Bière,63.536774,4.694836,17.214397,0.0,14.553986
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142664,351,2899,2015,28.0,0.0,0.0,230.0,0.0,-1356.0,6.0,-1120.0,0.0,0.0,0.0,749.0,1842.0,0.160034,0.0,0.010002,0.000000,Miscellanees,2903,Produits Vegetaux,2928,Miscellanees,-20.535715,-0.000000,-0.000000,-0.0,120.535713
142665,351,2899,2016,55.0,0.0,0.0,218.0,0.0,-1335.0,7.0,-1110.0,0.0,0.0,0.0,812.0,1867.0,0.150024,0.0,0.000000,0.000000,Miscellanees,2903,Produits Vegetaux,2928,Miscellanees,-19.639639,-0.000000,-0.000000,-0.0,119.639641
142666,351,2899,2017,60.0,0.0,0.0,207.0,0.0,-1481.0,7.0,-1267.0,0.0,0.0,0.0,890.0,2097.0,0.140015,0.0,0.000000,0.000000,Miscellanees,2903,Produits Vegetaux,2928,Miscellanees,-16.337807,-0.000000,-0.000000,-0.0,116.337807
142667,351,2899,2018,42.0,0.0,0.0,202.0,0.0,-1562.0,9.0,-1351.0,0.0,0.0,0.0,965.0,2274.0,0.140015,0.0,0.000000,0.000000,Miscellanees,2903,Produits Vegetaux,2928,Miscellanees,-14.951888,-0.000000,-0.000000,-0.0,114.951889


In [207]:
# # # ################################################################################################# Ancien code depalce pares calcul 

# # BilanAlimGeo=BilanAlim.merge(codePBA,how='left',on='CodeProduit').merge(geo, how='left')   

# # ColMove(BilanAlimGeo,'Zone',1)
# # ColMove(BilanAlimGeo,'Produit',2)
# # BilanAlimGeo.info()
# BilanAlimGeo  #144590 rows × 34 columns
# # # ################################################################################


# BilanAlim #142669 rows × 25 columns

# geo.columns
"""Index(['CodeZone', 'AlphaISO3', 'Zone', 'Année', 'Population', 'PopulationM','USDparHab', 'CodeContinent', 'continent', 'CodeZoneContinent','ZoneContinent'], dtype='object')"""

BilanAlimGeo=BilanAlim.merge(geo, how='left')  
ColMove(BilanAlimGeo,'Zone',1)
# ColMove(BilanAlimGeo,'Produit',2)
BilanAlimGeo.info()
BilanAlimGeo  #144590 rows × 34 columns



<class 'pandas.core.frame.DataFrame'>
Int64Index: 144590 entries, 0 to 144589
Data columns (total 39 columns):
 #   Column                                                             Non-Null Count   Dtype   
---  ------                                                             --------------   -----   
 0   CodeZone                                                           144590 non-null  int16   
 1   Zone                                                               143672 non-null  object  
 2   CodeProduit                                                        144590 non-null  int16   
 3   Année                                                              144590 non-null  int16   
 4   5072-Variation de stock                                            144590 non-null  float16 
 5   5123-Pertes                                                        144590 non-null  float32 
 6   5131-Traitement                                                    144590 non-null  float32 
 7   51

,CodeZone,Zone,CodeProduit,Année,5072-Variation de stock,5123-Pertes,5131-Traitement,5142-Nourriture,5154-Autres utilisations (non alimentaire),5170-Résidus,5171-Alimentation pour touristes,5301-Disponibilité intérieure,5511-Production,5521-Aliments pour animaux,5527-Semences,5611-Importations - Quantité,5911-Exportations - Quantité,645-Disponibilité alimentaire en quantité (kg/personne/an),664-Disponibilité alimentaire (Kcal/personne/jour),674-Disponibilité de protéines en quantité (g/personne/jour),684-Disponibilité de matière grasse en quantité (g/personne/jour),Produit,CodeOrigine,Origine,CodeCatProduits,GroupeProduits,%Nouriture,%Pertes,%AlimAnimale,%NonAlim,%NAutres,AlphaISO3,Population,PopulationM,USDparHab,CodeContinent,continent,CodeZoneContinent,ZoneContinent
0,1,Arménie,2511,2010,-43.0,8.0,16.0,370.0,0.0,100.0,0.0,577.0,183.0,61.0,22.0,351.0,0.0,128.625000,1026.0,30.656250,3.660156,Blé et produits,2903,Produits Vegetaux,2905,Céréales - Excl Bière,64.124779,1.386482,10.571924,0.0,23.916817,ARM,2.877000e+06,2.876953,115.9375,5300.0,Asie,5305.0,Asie occidentale
1,1,Arménie,2511,2011,21.0,9.0,16.0,384.0,0.0,88.0,0.0,581.0,224.0,65.0,20.0,378.0,0.0,133.625000,1061.0,31.656250,3.759766,Blé et produits,2903,Produits Vegetaux,2905,Céréales - Excl Bière,66.092941,1.549053,11.187608,0.0,21.170395,ARM,2.877000e+06,2.876953,121.3125,5300.0,Asie,5305.0,Asie occidentale
2,1,Arménie,2511,2012,0.0,11.0,17.0,367.0,0.0,222.0,0.0,733.0,243.0,92.0,23.0,490.0,1.0,127.312500,1013.0,30.156250,3.580078,Blé et produits,2903,Produits Vegetaux,2905,Céréales - Excl Bière,50.068211,1.500682,12.551160,0.0,35.879944,ARM,2.884000e+06,2.884766,111.1875,5300.0,Asie,5305.0,Asie occidentale
3,1,Arménie,2511,2013,30.0,16.0,17.0,379.0,0.0,101.0,0.0,636.0,312.0,93.0,30.0,355.0,1.0,130.750000,1040.0,30.984375,3.640625,Blé et produits,2903,Produits Vegetaux,2905,Céréales - Excl Bière,59.591190,2.515723,14.622643,0.0,23.270447,ARM,2.898000e+06,2.898438,112.7500,5300.0,Asie,5305.0,Asie occidentale
4,1,Arménie,2511,2014,42.0,30.0,16.0,406.0,0.0,46.0,0.0,639.0,338.0,110.0,32.0,344.0,1.0,139.375000,1107.0,33.000000,3.900391,Blé et produits,2903,Produits Vegetaux,2905,Céréales - Excl Bière,63.536774,4.694836,17.214397,0.0,14.553986,ARM,2.912000e+06,2.912109,113.5625,5300.0,Asie,5305.0,Asie occidentale
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144585,351,Chine,2899,2017,60.0,0.0,0.0,207.0,0.0,-1481.0,7.0,-1267.0,0.0,0.0,0.0,890.0,2097.0,0.140015,0.0,0.000000,0.000000,Miscellanees,2903,Produits Vegetaux,2928,Miscellanees,-16.337807,-0.000000,-0.000000,-0.0,116.337807,CHN,1.421022e+09,1421.000000,97.3750,5300.0,Asie,5302.0,Asie orientale
144586,351,Chine,2899,2018,42.0,0.0,0.0,202.0,0.0,-1562.0,9.0,-1351.0,0.0,0.0,0.0,965.0,2274.0,0.140015,0.0,0.000000,0.000000,Miscellanees,2903,Produits Vegetaux,2928,Miscellanees,-14.951888,-0.000000,-0.000000,-0.0,114.951889,CHN,1.459378e+09,1459.000000,103.0000,5300.0,Asie,5302.0,Asie orientale
144587,351,Chine,2899,2018,42.0,0.0,0.0,202.0,0.0,-1562.0,9.0,-1351.0,0.0,0.0,0.0,965.0,2274.0,0.140015,0.0,0.000000,0.000000,Miscellanees,2903,Produits Vegetaux,2928,Miscellanees,-14.951888,-0.000000,-0.000000,-0.0,114.951889,CHN,1.427648e+09,1428.000000,103.0000,5300.0,Asie,5302.0,Asie orientale
144588,351,Chine,2899,2019,75.0,0.0,0.0,174.0,0.0,-1400.0,9.0,-1218.0,0.0,0.0,0.0,1058.0,2200.0,0.119995,0.0,0.000000,0.000000,Miscellanees,2903,Produits Vegetaux,2928,Miscellanees,-14.285715,-0.000000,-0.000000,-0.0,114.285713,CHN,1.465634e+09,1466.000000,100.1875,5300.0,Asie,5302.0,Asie orientale


In [208]:
# BilanAlimGeo.info() #Int64Index: 144590   Category 
SearchNaN(BilanAlimGeo,2).CodeZone.unique()    #  ([151, 206, 214])   151	Antilles néerlandaises (ex) - 206	Soudan	   - 214 Chine, Taiwan Province de    / geo[geo.CodeZone==214]  OK !!!!! 

SearchNaN(BilanAlimGeo.iloc[:,[i for i in range(18)]],2)

# BilanAlimGeo.info()
# SearchNaN(BilanAlimGeo,2).CodeZone.sort_values(ascending=True,na_position='first').unique()

,CodeZone,Zone,CodeProduit,Année,5072-Variation de stock,5123-Pertes,5131-Traitement,5142-Nourriture,5154-Autres utilisations (non alimentaire),5170-Résidus,5171-Alimentation pour touristes,5301-Disponibilité intérieure,5511-Production,5521-Aliments pour animaux,5527-Semences,5611-Importations - Quantité,5911-Exportations - Quantité,645-Disponibilité alimentaire en quantité (kg/personne/an)
87041,151,NaN,2761,2010,0.000000,0.0,0.0,1.02,0.0,0.0,0.0,1.02,0.00,0.0,0.0,1.02,0.00,0.000000
87042,151,NaN,2762,2010,0.000000,0.0,0.0,0.96,0.0,0.0,0.0,0.96,0.00,0.0,0.0,1.01,0.05,0.000000
87043,151,NaN,2763,2010,-0.199951,0.0,0.0,1.05,4.6,0.0,0.0,5.65,18.76,0.0,0.0,0.84,14.15,0.000000
87044,151,NaN,2764,2010,0.000000,0.0,0.0,1.67,0.7,0.0,0.0,2.37,0.75,0.0,0.0,1.67,0.04,0.000000
87045,151,NaN,2765,2010,0.000000,0.0,0.0,0.57,0.2,0.0,0.0,0.78,0.00,0.0,0.0,0.78,0.00,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120938,214,NaN,2899,2015,0.000000,0.0,0.0,57.00,0.0,0.0,0.0,57.00,0.00,0.0,0.0,175.00,118.00,2.439453
120939,214,NaN,2899,2016,0.000000,0.0,0.0,54.00,0.0,0.0,0.0,54.00,0.00,0.0,0.0,181.00,127.00,2.279297
120940,214,NaN,2899,2017,0.000000,0.0,0.0,40.00,0.0,0.0,0.0,40.00,0.00,0.0,0.0,181.00,140.00,1.709961
120941,214,NaN,2899,2018,0.000000,0.0,0.0,23.00,0.0,0.0,0.0,23.00,0.00,0.0,0.0,194.00,171.00,0.950195


In [209]:
geo[geo.CodeZone==214]

,CodeZone,AlphaISO3,Zone,Année,Population,PopulationM,USDparHab,CodeContinent,continent,CodeZoneContinent,ZoneContinent
3095,214,TWN,"Chine, Taiwan Province de",0,0.0,0.0,0.0,5300,Asie,5302,Asie orientale


### Bilan Alim Final

In [210]:
BilanAlimGeo
# BilanAlim.info(memory_usage=True)
# SearchNaN(BilanAlim,1) # OK pas de NaN

,CodeZone,Zone,CodeProduit,Année,5072-Variation de stock,5123-Pertes,5131-Traitement,5142-Nourriture,5154-Autres utilisations (non alimentaire),5170-Résidus,5171-Alimentation pour touristes,5301-Disponibilité intérieure,5511-Production,5521-Aliments pour animaux,5527-Semences,5611-Importations - Quantité,5911-Exportations - Quantité,645-Disponibilité alimentaire en quantité (kg/personne/an),664-Disponibilité alimentaire (Kcal/personne/jour),674-Disponibilité de protéines en quantité (g/personne/jour),684-Disponibilité de matière grasse en quantité (g/personne/jour),Produit,CodeOrigine,Origine,CodeCatProduits,GroupeProduits,%Nouriture,%Pertes,%AlimAnimale,%NonAlim,%NAutres,AlphaISO3,Population,PopulationM,USDparHab,CodeContinent,continent,CodeZoneContinent,ZoneContinent
0,1,Arménie,2511,2010,-43.0,8.0,16.0,370.0,0.0,100.0,0.0,577.0,183.0,61.0,22.0,351.0,0.0,128.625000,1026.0,30.656250,3.660156,Blé et produits,2903,Produits Vegetaux,2905,Céréales - Excl Bière,64.124779,1.386482,10.571924,0.0,23.916817,ARM,2.877000e+06,2.876953,115.9375,5300.0,Asie,5305.0,Asie occidentale
1,1,Arménie,2511,2011,21.0,9.0,16.0,384.0,0.0,88.0,0.0,581.0,224.0,65.0,20.0,378.0,0.0,133.625000,1061.0,31.656250,3.759766,Blé et produits,2903,Produits Vegetaux,2905,Céréales - Excl Bière,66.092941,1.549053,11.187608,0.0,21.170395,ARM,2.877000e+06,2.876953,121.3125,5300.0,Asie,5305.0,Asie occidentale
2,1,Arménie,2511,2012,0.0,11.0,17.0,367.0,0.0,222.0,0.0,733.0,243.0,92.0,23.0,490.0,1.0,127.312500,1013.0,30.156250,3.580078,Blé et produits,2903,Produits Vegetaux,2905,Céréales - Excl Bière,50.068211,1.500682,12.551160,0.0,35.879944,ARM,2.884000e+06,2.884766,111.1875,5300.0,Asie,5305.0,Asie occidentale
3,1,Arménie,2511,2013,30.0,16.0,17.0,379.0,0.0,101.0,0.0,636.0,312.0,93.0,30.0,355.0,1.0,130.750000,1040.0,30.984375,3.640625,Blé et produits,2903,Produits Vegetaux,2905,Céréales - Excl Bière,59.591190,2.515723,14.622643,0.0,23.270447,ARM,2.898000e+06,2.898438,112.7500,5300.0,Asie,5305.0,Asie occidentale
4,1,Arménie,2511,2014,42.0,30.0,16.0,406.0,0.0,46.0,0.0,639.0,338.0,110.0,32.0,344.0,1.0,139.375000,1107.0,33.000000,3.900391,Blé et produits,2903,Produits Vegetaux,2905,Céréales - Excl Bière,63.536774,4.694836,17.214397,0.0,14.553986,ARM,2.912000e+06,2.912109,113.5625,5300.0,Asie,5305.0,Asie occidentale
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144585,351,Chine,2899,2017,60.0,0.0,0.0,207.0,0.0,-1481.0,7.0,-1267.0,0.0,0.0,0.0,890.0,2097.0,0.140015,0.0,0.000000,0.000000,Miscellanees,2903,Produits Vegetaux,2928,Miscellanees,-16.337807,-0.000000,-0.000000,-0.0,116.337807,CHN,1.421022e+09,1421.000000,97.3750,5300.0,Asie,5302.0,Asie orientale
144586,351,Chine,2899,2018,42.0,0.0,0.0,202.0,0.0,-1562.0,9.0,-1351.0,0.0,0.0,0.0,965.0,2274.0,0.140015,0.0,0.000000,0.000000,Miscellanees,2903,Produits Vegetaux,2928,Miscellanees,-14.951888,-0.000000,-0.000000,-0.0,114.951889,CHN,1.459378e+09,1459.000000,103.0000,5300.0,Asie,5302.0,Asie orientale
144587,351,Chine,2899,2018,42.0,0.0,0.0,202.0,0.0,-1562.0,9.0,-1351.0,0.0,0.0,0.0,965.0,2274.0,0.140015,0.0,0.000000,0.000000,Miscellanees,2903,Produits Vegetaux,2928,Miscellanees,-14.951888,-0.000000,-0.000000,-0.0,114.951889,CHN,1.427648e+09,1428.000000,103.0000,5300.0,Asie,5302.0,Asie orientale
144588,351,Chine,2899,2019,75.0,0.0,0.0,174.0,0.0,-1400.0,9.0,-1218.0,0.0,0.0,0.0,1058.0,2200.0,0.119995,0.0,0.000000,0.000000,Miscellanees,2903,Produits Vegetaux,2928,Miscellanees,-14.285715,-0.000000,-0.000000,-0.0,114.285713,CHN,1.465634e+09,1466.000000,100.1875,5300.0,Asie,5302.0,Asie orientale


## Global Alim 

### ZZZZZZZZZZZZZZZZZ SecuALim OK 4074 rows × 5 columns

In [211]:
SecuAlim

ListeProduit,CodeZone,AnnéeREF,210011-Nombre de personnes sous-alimentées (millions) (moyenne sur 3 ans),21056-Besoins énergétiques alimentaires minimaux (kcal/personne/jour),21057-Besoins énergétiques alimentaires moyens (kcal/personne/jour)
0,1,2000,0,1839,2378
1,1,2001,0.8,1849,2394
2,1,2002,0.7,1858,2407
3,1,2003,0.6,1865,2418
4,1,2004,0.5,1871,2428
...,...,...,...,...,...
4069,351,2016,0,1883,2445
4070,351,2017,0,1883,2444
4071,351,2018,0,1883,2442
4072,351,2019,0,1882,2441


### Bilan Alim 

In [247]:
# Merge pour faire apparaitre le texte correspondant aux fk Code.....

#Detail des df  : 
#BilanAlim.columns
""" Index(['CodeZone', 'CodeProduit', 'Année', '5072-Variation de stock', '5123-Pertes', '5131-Traitement',...], dtype='object', name='Elément')"""
# CodeProduitBA.columns 
"""Index(['CodeProduit', 'Produit', 'CodeOrigine', 'Origine', 'CodeCatProduits','GroupeProduits', 'Coefficient', 'Description'], dtype='object')"""

codePBA=CodeProduitBA.iloc[:,[i for i in range(6)]]  # selection des 6 premiere colonnes  (tauf sauf 'Coefficient', 'Description' )

################################################ Merge sans le Geo #######################################################
# BilanAlim=BilanAlim.merge(codePBA,how='left',on='CodeProduit') #  Ok 142669 rows × 20 columns => 142669 rows × 25 columns
# BilanAlim
#############################################################################################################################

In [213]:
codePBA=CodeProduitBA.iloc[:,[i for i in range(6)]]  # selection des 6 premiere colonnes  (tauf sauf 'Coefficient', 'Description' )

############################################### Merge sans le Geo #######################################################
TA=BilanAlim.merge(codePBA,how='left',on='CodeProduit') #  Ok 142669 rows × 20 columns => 142669 rows × 25 columns
TA
#############################################################################################################################

,CodeZone,CodeProduit,Année,5072-Variation de stock,5123-Pertes,5131-Traitement,5142-Nourriture,5154-Autres utilisations (non alimentaire),5170-Résidus,5171-Alimentation pour touristes,5301-Disponibilité intérieure,5511-Production,5521-Aliments pour animaux,5527-Semences,5611-Importations - Quantité,5911-Exportations - Quantité,645-Disponibilité alimentaire en quantité (kg/personne/an),664-Disponibilité alimentaire (Kcal/personne/jour),674-Disponibilité de protéines en quantité (g/personne/jour),684-Disponibilité de matière grasse en quantité (g/personne/jour),Produit_x,CodeOrigine_x,Origine_x,CodeCatProduits_x,GroupeProduits_x,%Nouriture,%Pertes,%AlimAnimale,%NonAlim,%NAutres,Produit_y,CodeOrigine_y,Origine_y,CodeCatProduits_y,GroupeProduits_y
0,1,2511,2010,-43.0,8.0,16.0,370.0,0.0,100.0,0.0,577.0,183.0,61.0,22.0,351.0,0.0,128.625000,1026.0,30.656250,3.660156,Blé et produits,2903,Produits Vegetaux,2905,Céréales - Excl Bière,64.124779,1.386482,10.571924,0.0,23.916817,Blé et produits,2903,Produits Vegetaux,2905,Céréales - Excl Bière
1,1,2511,2011,21.0,9.0,16.0,384.0,0.0,88.0,0.0,581.0,224.0,65.0,20.0,378.0,0.0,133.625000,1061.0,31.656250,3.759766,Blé et produits,2903,Produits Vegetaux,2905,Céréales - Excl Bière,66.092941,1.549053,11.187608,0.0,21.170395,Blé et produits,2903,Produits Vegetaux,2905,Céréales - Excl Bière
2,1,2511,2012,0.0,11.0,17.0,367.0,0.0,222.0,0.0,733.0,243.0,92.0,23.0,490.0,1.0,127.312500,1013.0,30.156250,3.580078,Blé et produits,2903,Produits Vegetaux,2905,Céréales - Excl Bière,50.068211,1.500682,12.551160,0.0,35.879944,Blé et produits,2903,Produits Vegetaux,2905,Céréales - Excl Bière
3,1,2511,2013,30.0,16.0,17.0,379.0,0.0,101.0,0.0,636.0,312.0,93.0,30.0,355.0,1.0,130.750000,1040.0,30.984375,3.640625,Blé et produits,2903,Produits Vegetaux,2905,Céréales - Excl Bière,59.591190,2.515723,14.622643,0.0,23.270447,Blé et produits,2903,Produits Vegetaux,2905,Céréales - Excl Bière
4,1,2511,2014,42.0,30.0,16.0,406.0,0.0,46.0,0.0,639.0,338.0,110.0,32.0,344.0,1.0,139.375000,1107.0,33.000000,3.900391,Blé et produits,2903,Produits Vegetaux,2905,Céréales - Excl Bière,63.536774,4.694836,17.214397,0.0,14.553986,Blé et produits,2903,Produits Vegetaux,2905,Céréales - Excl Bière
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142664,351,2899,2015,28.0,0.0,0.0,230.0,0.0,-1356.0,6.0,-1120.0,0.0,0.0,0.0,749.0,1842.0,0.160034,0.0,0.010002,0.000000,Miscellanees,2903,Produits Vegetaux,2928,Miscellanees,-20.535715,-0.000000,-0.000000,-0.0,120.535713,Miscellanees,2903,Produits Vegetaux,2928,Miscellanees
142665,351,2899,2016,55.0,0.0,0.0,218.0,0.0,-1335.0,7.0,-1110.0,0.0,0.0,0.0,812.0,1867.0,0.150024,0.0,0.000000,0.000000,Miscellanees,2903,Produits Vegetaux,2928,Miscellanees,-19.639639,-0.000000,-0.000000,-0.0,119.639641,Miscellanees,2903,Produits Vegetaux,2928,Miscellanees
142666,351,2899,2017,60.0,0.0,0.0,207.0,0.0,-1481.0,7.0,-1267.0,0.0,0.0,0.0,890.0,2097.0,0.140015,0.0,0.000000,0.000000,Miscellanees,2903,Produits Vegetaux,2928,Miscellanees,-16.337807,-0.000000,-0.000000,-0.0,116.337807,Miscellanees,2903,Produits Vegetaux,2928,Miscellanees
142667,351,2899,2018,42.0,0.0,0.0,202.0,0.0,-1562.0,9.0,-1351.0,0.0,0.0,0.0,965.0,2274.0,0.140015,0.0,0.000000,0.000000,Miscellanees,2903,Produits Vegetaux,2928,Miscellanees,-14.951888,-0.000000,-0.000000,-0.0,114.951889,Miscellanees,2903,Produits Vegetaux,2928,Miscellanees


In [214]:
SecuAlim.info() #4074 entries  et pas de NaN

#NaN
# SearchNaN(SecuAlim,1)
SecuAlim.isnull().any()

#Valeur 0
# # SearchListe(SecuAlim,[0])
# get_different_rows(SearchListe(SecuAlim,[0]),SecuAlim)
# print(len(SecuAlim),len(SearchListe(SecuAlim,[0])))
# SecuAlim.duplicated().describe ()  # Pas de valeur duplicquées 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4074 entries, 0 to 4073
Data columns (total 5 columns):
 #   Column                                                                     Non-Null Count  Dtype 
---  ------                                                                     --------------  ----- 
 0   CodeZone                                                                   4074 non-null   int64 
 1   AnnéeREF                                                                   4074 non-null   int64 
 2   210011-Nombre de personnes sous-alimentées (millions) (moyenne sur 3 ans)  4074 non-null   object
 3   21056-Besoins énergétiques alimentaires minimaux (kcal/personne/jour)      4074 non-null   object
 4   21057-Besoins énergétiques alimentaires moyens (kcal/personne/jour)        4074 non-null   object
dtypes: int64(2), object(3)
memory usage: 159.3+ KB


ListeProduit
CodeZone                                                                     False
AnnéeREF                                                                     False
210011-Nombre de personnes sous-alimentées (millions) (moyenne sur 3 ans)    False
21056-Besoins énergétiques alimentaires minimaux (kcal/personne/jour)        False
21057-Besoins énergétiques alimentaires moyens (kcal/personne/jour)          False
dtype: bool

In [215]:
SecuAlim.CodeZone.value_counts().unique() # Verif duplication #21
SecuAlim.Année.value_counts().unique() # Verif duplication #194

SearchListe(SecuAlim,[0]) ## 4074 entries dont 1882 rows with [0]

AttributeError: ignored

##### Secu Alim Code Recherhce Bug 

In [ ]:
#  df=SecuAlim
#  ListeCol=df.columns
# for col in ListeCol: 
#   print(col,'Val unique :\n',df[col].value_counts().unique(),'\n______________________\n') # resumé
#   # print(col,'Val unique :\n',df[col].value_counts(),'\n______________________\n') # detail

In [ ]:
def UniqueNb(df,mode):
  listeCol=df.columns
  if mode==0:
    for col in ListeCol: 
      return  print(col,'Val unique :\n',df[col].value_counts().unique(),'\n______________________\n') # resumé
        # return print(col,'Val unique :\n',df[col].value_counts(),'\n______________________\n') # detail
  else :
      for col in ListeCol: 
        # print(col,'Val unique :\n',df[col].value_counts().unique(),'\n______________________\n') # resumé
        return print(col,'Val unique :\n',df[col].value_counts(),'\n______________________\n') # detail


#   #DEBUG 
#  f=SecuAlim
# for col in ListeCol: 
#   # print(col,'Val unique :\n',df[col].value_counts().unique(),'\n______________________\n') # resumé
#   print(col,'Val unique :\n',df[col].value_counts(),'\n______________________\n') # detail

In [ ]:
 f=SecuAlim
for col in ListeCol: 
  # print(col,'Val unique :\n',df[col].value_counts().unique(),'\n______________________\n') # resumé
  print(col,'Val unique :\n',df[col].value_counts(),'\n______________________\n') # detail

In [ ]:
UniqueNb(geo,1)

In [ ]:
geo


In [ ]:
BilanAlim.columns

In [ ]:
liste=BilanAlim.columns
liste=liste[3:] # retirer les 3 premieree colonnes 'CodeZone', 'CodeProduit', 'Année'
dl=BilanAlim.groupby(["CodeZone","Année"])[liste].sum()
dl.reset_index(inplace=True) # convertir l'index en colonnes du df 

dr=df=SecuAlim.merge(geo, how='left',on=['CodeZone','Année'])

GlobalAlim=dl.merge(dr, how='left',on=['CodeZone','Année'])


### Debug des NaN 

temp=GlobalAlim
# temp.isnull().any()
# SearchNaN(temp,2)
temp[temp.Zone.isna()]


# BAckup Merge 
# df=SecuAlim.merge(geo, how='left',on=['CodeZone','Année'])
# df

In [ ]:
GlobalAlim.columns

In [ ]:
GlobalAlim.info()

In [ ]:
# df=GlobalAlim

# ListeCol=df.columns

# for col in ListeCol:
#   print(col,':',len(df[col]),'\n',df[col].sort_values(ascending=True,na_position='first').unique(),'\n______________________________\n')

In [ ]:
liste=BilanAlim.columns
liste=liste[3:] # retirer les 3 premieree colonnes 'CodeZone', 'CodeProduit', 'Année'

In [ ]:
dtypeCat2Obj(GlobalAlim)
GlobalAlim=GlobalAlim.fillna(0)
# reduce_mem_usage(GlobalAlim)

ColMove(GlobalAlim,'Zone',2)


# Estiùmation du nombre de kCal nécessaire par Pays
# Besoin Moyen  (kcal/personne/jour)	21057-Besoins énergétiques alimentaires moyens ) x PopPulation 
GlobalAlim['Kcal_MinAnnuel']=GlobalAlim['21056-Besoins énergétiques alimentaires minimaux (kcal/personne/jour)']*GlobalAlim['Population']*365

In [ ]:
geo

# Data Set [OC]

In [ ]:
#  Importer les données en Pandas dataframe
#aideDataPATH='/content/drive/MyDrive/DataSc/P4VA/'
DataPATH='https://raw.githubusercontent.com/vincent-arese/OC-P04VA/main/data/'

aide=pd.read_csv(DataPATH+'aide_alimentaire.csv')             # OK Aide Alimentaire
dispo=pd.read_csv(DataPATH+'dispo_alimentaire.csv')           # OK Bilan Alimentaire
pop=pd.read_csv(DataPATH+'population.csv')                    # OK Population Mondiale
snut=pd.read_csv(DataPATH+'sous_nutrition.csv')
# geocode=pd.read_csv(DataPATH+'CodePaysMonde.csv', sep = ';')

## Comparaison des Data Se FAO & OC

In [ ]:
pop.Année.unique()

In [ ]:
aide.head(1)

In [ ]:
aide.Année.unique()

In [ ]:
dispo.head(1)

In [ ]:
# Code Buggé car changement  de csv 



# FAO=BilanAlim.Produit.unique()
# OC=dispo.Produit.unique()
# print('FAO:',len(FAO),'\nOC:',len(OC),'\nIntersect:',(((len(np.intersect1d(FAO, OC))/len(FAO))*100)),'%',np.intersect1d(FAO, OC))

In [ ]:
BilanAlim.info()

In [ ]:
# BilanAlim[BilanAlim.Année==2017 & BilanAlim.Zone=='Afghanistan']

In [ ]:
pop.head(1)

In [ ]:
snut.head(1)

In [ ]:
# pop.Zone=pop.Zone.astype("category")
# pop.memory_usage(deep=True)

In [ ]:
# !cat /proc/meminfo # Voir RAM

# Panorama de l’état de la malnutrition dans le monde (2017)

## 1 Nombre de personnes vistime de sous-nutrition dans le monde



Nombre de personnes victime de sous-nutrition dans le monde
Proportion de personnes en état de sous-nutrition (Q1)
Map monde ( Nb personnes / %pop sous nutrition)  (Q5)
Tableau Sous-nutrition (Total –  Pays   total / rankin tip 20 / 20/80) (Q5)
Detail 2017 
•	Q5 les pays pour lesquels la proportion de personnes sous-alimentées est la plus forte en 2017,


## 2 Aide Alimentaire

Aide Alimentaire 
•	Q6 PAYS eux qui ont le plus bénéficié d’aide depuis 2013,
Cumul  2016 
G/hab distribé 


## Bilan Alimentaire mondial

**Disponibilité Alimetaire Mondiale **
•	Q2 le nombre théorique de personnes qui pourraient être nourries. Tu devrais pouvoir calculer ça à partir de la disponibilité alimentaire mondiale ;
•	Q3 Dispo produit vegetal  2017
•	Q4 Dispo produit Animal  2017 
•	Disport int  / alimen,taion Animale / alimen,taion Humaine/pertes
Q9 / céreéales
**Focus Pays **
•	Q7 PAYS ceux ayant le plus/le moins de disponibilité/habitant
Part Alim Animale/Humaine / pertes (% et total Kcal Perdues)
Deficit Kcal  par Pays   (ous-alim  / alime,taion moyenne) 

**Cas particuliers **
Q9 / céreéales
Q10 Manoc 


# A faire 

---
SecuAlim  : Order by Pays (desc) et somme cimulée pour 20 /80 


AideAlimentaire/geo 
Cumul Aide Alime  Période 
2Fichier par PAYs 
Detail   g/personne /jour  (1t=10e+6g - Pop: Millionhab)
Grapge pour voir la propotion Tonnage par Continent / par Pays 

Bilan Alimentaire mondial
Nombre de personnes pouvant être nouries 
BilanAlim. Group by PAYS  (Animal /Vegetal) - 
BilanAlim .DispoInt(t ou milliers tonnes) /


Bilan Alim Calcul 
Code Élément5301
ÉlémentDisponibilité intérieure
UnitéMilliers de tonnes
Description Production + importations - exportations + évolution des stocks (augmentation ou diminution) = disponibilités servant à une utilisation intérieure.Il existe plusieurs définitions des disponibilités et, dans la pratique, plusieurs notions entrent en jeu. Les éléments concernés sont la production, les importations, les exportations et l'évolution des stocks (augmentation ou diminution). La production, les importations et l'évolution des stocks (à la hausse ou à la baisse) représentent, sans conteste, des éléments constitutifs des disponibilités. Source: Division de la statistique (FAO)

DispoInt 
%Nouriture = Nourriture/DispoInt
%Pertes= Perteres/DispoInt
%AlimAnimale=AlimAnia/DispoiNt 

Travailler avc un groupby Groupe Produits

https://mail.google.com/chat/u/0/#chat/dm/_PihIoAAAAE



 Marc
pour l’année 2017 :
•	Q1 : la proportion de personnes en état de sous-nutrition ;
•	Q2 le nombre théorique de personnes qui pourraient être nourries. Tu devrais pouvoir calculer ça à partir de la disponibilité alimentaire mondiale ;
•	Q3 idem pour la disponibilité alimentaire des produits végétaux ;
•	Q4 l’utilisation de la disponibilité intérieure, en particulier la part qui est attribuée à l’alimentation animale, celle qui est perdue et celle qui est concrètement utilisée pour l'alimentation humaine. Je crois que Julien avait trouvé un moyen de facilement calculer ces proportions.
Mélanie, une autre chercheuse de l’équipe
Une étude un peu plus fine pour chacun des pays.
•	Q5 les pays pour lesquels la proportion de personnes sous-alimentées est la plus forte en 2017,
•	Q6 PAYS eux qui ont le plus bénéficié d’aide depuis 2013,
•	Q7 PAYS ceux ayant le plus/le moins de disponibilité/habitant
•	Q8 toutes les infos que tu trouverais utiles pour mettre en relief les pays qui semblent être le plus en difficulté, au niveau alimentaire.
Julien : 
Q9 : ors de mes recherches, j’ai découvert des chiffres intéressants sur l’utilisation des céréales, notamment la répartition entre l’alimentation humaine (colonne Nourriture) et l’alimentation pour animaux
●	Q10 Enfin, je me souviens d’un exemple étonnant d’utilisation du manioc par la Thaïlande aux égards de la proportion de personnes en sous-nutrition. Il serait intéressant d’aller à nouveau y jeter un coup d’œil, de mémoire ça concernait l’exportation par rapport à la production.


Nombre de personnes victime de sous-nutrition dans le monde
Proportion de personnes en état de sous-nutrition (Q1)
Map monde ( Nb personnes / %pop sous nutrition)  (Q5)
Tableau Sous-nutrition (Total –  Pays   total / rankin tip 20 / 20/80) (Q5)
Detail 2017 
•	Q5 les pays pour lesquels la proportion de personnes sous-alimentées est la plus forte en 2017,


Aide Alimentaire 
•	Q6 PAYS eux qui ont le plus bénéficié d’aide depuis 2013,
Cumul  2016 
G/hab distribé 

Bilan Alimentaire mondial

Disponibilité Alimetaire Mondiale 
•	Q2 le nombre théorique de personnes qui pourraient être nourries. Tu devrais pouvoir calculer ça à partir de la disponibilité alimentaire mondiale ;
•	Q3 Dispo produit vegetal  2017
•	Q4 Dispo produit Animal  2017 
•	Disport int  / alimen,taion Animale / alimen,taion Humaine/pertes
Q9 / céreéales
Focus Pays 
•	Q7 PAYS ceux ayant le plus/le moins de disponibilité/habitant
Part Alim Animale/Humaine / pertes (% et total Kcal Perdues)
Deficit Kcal  par Pays   (ous-alim  / alime,taion moyenne) 

Q9 / céreéales
Q10 Manoc 


#DEBUG

In [ ]:
def UniqueNb(df,mode):
  listeCol=df.columns
  if mode==0:
    for col in ListeCol: 
      return  print(col,'Val unique :\n',df[col].value_counts().unique(),'\n______________________\n') # resumé
        # return print(col,'Val unique :\n',df[col].value_counts(),'\n______________________\n') # detail
  else :
      for col in ListeCol: 
        # print(col,'Val unique :\n',df[col].value_counts().unique(),'\n______________________\n') # resumé
        return print(col,'Val unique :\n',df[col].value_counts(),'\n______________________\n') # detail


#   #DEBUG 
#  f=SecuAlim
# for col in ListeCol: 
#   # print(col,'Val unique :\n',df[col].value_counts().unique(),'\n______________________\n') # resumé
#   print(col,'Val unique :\n',df[col].value_counts(),'\n______________________\n') # detail

In [ ]:
temp=SecuAlimGeo.dropna(thresh=7)
listeAN=temp.Année.unique()
for a in listeAN:
  print(a,': ',len(temp.query("Année=="+str(a))))

In [ ]:
A=temp.CodeZone.unique()
B=SecuAlim.CodeZone.unique()
LostElement(A,B)

In [ ]:
print(len(SecuAlim),len(SecuAlimGeo))

get_different_rows(SecuAlim, SecuAlimGeo)

In [ ]:
SecuAlimGeo.query("Année==2000")

In [ ]:
SecuAlimGeo.query("Année==2020")

In [ ]:
A=SecuAlimGeo.CodeZone.unique()
B=SecuAlim.CodeZone.unique()
LostElement(A,B)

In [ ]:
# DEBUG
listeAN=SecuAlimGeo.Année.unique()
for a in listeAN:
  print(a,': ',len(SecuAlimGeo.query("Année=="+str(a))))

In [ ]:
# #DEBUG
# SecuAlim.info(memory_usage=True)
# SearchNaN(SecuAlim,1) # OK pas de NaN